In [2]:
import pandas as pd
import numpy as np
import time
from google.colab import files, drive
import re
from typing import Dict, List, Optional, Tuple

# MVP Table

In [3]:
# Skip the scraping and use the already downloaded
drive.mount('/content/drive')  # follow the auth link

mvp_path = '/content/drive/MyDrive/CS3540/Project/mvp_df.csv'
mvp_df = pd.read_csv(mvp_path)
mvp_df.head()

Mounted at /content/drive


,Unnamed: 0,Unnamed: 0_level_0 Rk,Unnamed: 1_level_0 Pos,Unnamed: 2_level_0 Player,Unnamed: 3_level_0 Tm,Unnamed: 4_level_0 Vote Pts,Unnamed: 5_level_0 1st Place,Unnamed: 6_level_0 Share,Games G,Games GS,...,Unnamed: 21_level_0 Sk,Unnamed: 22_level_0 Int,Year,Unnamed: 4_level_0 Votes,Unnamed: 5_level_0 Share,Unnamed: 19_level_0 Solo,Unnamed: 20_level_0 Sk,Unnamed: 21_level_0 Int,Unnamed: 19_level_0 Sk,Unnamed: 20_level_0 Int
0,0,1,QB,Josh Allen,Buffalo Bills,378.0,27.0,77.1%,17,17,...,0.0,0.0,2024,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,2,QB,Lamar Jackson,Baltimore Ravens,352.0,22.0,71.8%,17,17,...,0.0,0.0,2024,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,3,RB,Saquon Barkley,Philadelphia Eagles,117.0,0.0,23.9%,16,16,...,0.0,0.0,2024,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,4,QB,Joe Burrow,Cincinnati Bengals,82.0,0.0,16.7%,17,17,...,0.0,0.0,2024,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,5,QB,Jared Goff,Detroit Lions,47.0,0.0,9.6%,17,17,...,0.0,0.0,2024,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
year = 2024
mvp_dfs = []

while year >= 1980:
    url = 'https://www.pro-football-reference.com/awards/awards_' + str(year) + '.htm'

    try:
        # Get all tables from the page
        tables = pd.read_html(url)

        # tables[0] is MVP voting
        mvp_table = tables[0]

        # Check if columns are multi-level and handle accordingly
        if isinstance(mvp_table.columns, pd.MultiIndex):
            # For multi-level columns, add year as a regular column after flattening
            mvp_table.columns = [' '.join(str(col).strip() for col in multi_col if str(col) != 'nan')
                                for multi_col in mvp_table.columns]
            mvp_table['Year'] = year
        else:
            # For regular columns, add year normally
            mvp_table['Year'] = year

        mvp_dfs.append(mvp_table)
        print(f"Scraped {year} - Shape: {mvp_table.shape}")

    except Exception as e:
        print(f"Failed to scrape {year}: {e}")

    time.sleep(3)
    year -= 1

# Combine all dataframes
mvp_df = pd.concat(mvp_dfs, ignore_index=True)
print(f"Total rows: {len(mvp_df)}")

# Check if Year column exists and show some info
print("\nColumns in final dataframe:")
print(mvp_df.columns.tolist())

if 'Year' in mvp_df.columns:
    print(f"\nYear column added successfully")
    print(f"Years covered: {mvp_df['Year'].min()} - {mvp_df['Year'].max()}")
    print(f"Rows per year: {mvp_df['Year'].value_counts().sort_index()}")
else:
    print("Year column not found")
    print("Sample of dataframe:")
    print(mvp_df.head())

Scraped 2024 - Shape: (11, 24)
Scraped 2023 - Shape: (10, 24)


KeyboardInterrupt: 

In [ ]:
mvp_df.to_csv('mvp_df.csv')
files.download('mvp_df.csv')

# MVP Cleaning

In [ ]:
# Skip cleaning code
mvp_clean_path = '/content/drive/MyDrive/CS3540/Project/mvp_df_clean.csv'
mvp_df_clean = pd.read_csv(mvp_clean_path)
mvp_df_clean.head()

,Unnamed: 0,Year,Rank,Player,Team,Position,Votes,G,GS,Pass_Cmp,...,Pass_Int,Rush_Att,Rush_Yds,Rush_TD,Rec_Rec,Rec_Yds,Rec_TD,Def_Solo,Def_Sk,Def_Int
0,0,2024,1,Josh Allen,Buffalo Bills,QB,90.0,17,17,307,...,6,102,531,12,0,7,1,0.0,0.0,0.0
1,1,2024,2,Lamar Jackson,Baltimore Ravens,QB,84.0,17,17,316,...,4,139,915,4,0,0,0,0.0,0.0,0.0
2,2,2024,3,Saquon Barkley,Philadelphia Eagles,RB,28.0,16,16,0,...,0,345,2005,13,33,278,2,0.0,0.0,0.0
3,3,2024,4,Joe Burrow,Cincinnati Bengals,QB,20.0,17,17,460,...,9,42,201,2,0,0,0,0.0,0.0,0.0
4,4,2024,5,Jared Goff,Detroit Lions,QB,11.0,17,17,390,...,12,35,56,0,1,7,1,0.0,0.0,0.0


In [ ]:
mvp_df_clean = mvp_df.copy()

# Drop unnecessary columns
mvp_df_clean = mvp_df_clean.drop(columns = ["Unnamed: 0"], errors = "ignore")

# Rename columns
column_mapping = {
    "Unnamed: 0_level_0 Rk": "Rank",
    "Unnamed: 1_level_0 Pos": "Position",
    "Unnamed: 2_level_0 Player": "Player",
    "Unnamed: 3_level_0 Tm": "Team",
    "Unnamed: 4_level_0 Vote Pts": "VotePoints_1",
    "Unnamed: 4_level_0 Votes": "VotePoints_2",
    "Unnamed: 5_level_0 1st Place": "FirstPlaceVotes",
    "Unnamed: 6_level_0 Share": "VoteShare_1",
    "Unnamed: 5_level_0 Share": "VoteShare_2",
    "Games G": "G",
    "Games GS": "GS",
    "Passing Cmp": "Pass_Cmp",
    "Passing Att": "Pass_Att",
    "Passing Yds": "Pass_Yds",
    "Passing TD": "Pass_TD",
    "Passing Int": "Pass_Int",
    "Rushing Att": "Rush_Att",
    "Rushing Yds": "Rush_Yds",
    "Rushing TD": "Rush_TD",
    "Receiving Rec": "Rec_Rec",
    "Receiving Yds": "Rec_Yds",
    "Receiving TD": "Rec_TD"
}
mvp_df_clean = mvp_df_clean.rename(columns = column_mapping)

# Consolidate voting
vote_points_cols = [c for c in ["VotePoints_1", "VotePoints_2"] if c in mvp_df_clean.columns]
if vote_points_cols:
    mvp_df_clean["VotePoints"] = pd.to_numeric(mvp_df_clean[vote_points_cols[0]], errors="coerce")
    for col in vote_points_cols[1:]:
        mvp_df_clean["VotePoints"] = mvp_df_clean["VotePoints"].fillna(pd.to_numeric(mvp_df_clean[col], errors = "coerce"))
    mvp_df_clean = mvp_df_clean.drop(columns = vote_points_cols)

# Convert FirstPlaceVotes to numeric
if "FirstPlaceVotes" in mvp_df_clean.columns:
    mvp_df_clean["FirstPlaceVotes"] = pd.to_numeric(mvp_df_clean["FirstPlaceVotes"], errors = "coerce")

# Unified votes column
if "VotePoints" in mvp_df_clean.columns and "Year" in mvp_df_clean.columns:
    # 2022 and after: convert point totals back to estimated ballot counts
    # Point system: 10 pts (1st), 5 pts (2nd), 3 pts (3rd), 2 pts (4th), 1 pt (5th)
    # Average points per ballot = (10+5+3+2+1)/5 = 4.2 points per ballot
    # Divide total points by 4.2 to estimate total ballots received

    # Handle the conversion with NaN values and ensure minimum of 1 vote
    votes_2022_plus = mvp_df_clean["VotePoints"] / 4.2
    votes_2022_plus = np.where(pd.isna(votes_2022_plus), np.nan, np.round(votes_2022_plus))
    # Ensure anyone with points gets at least 1 vote
    votes_2022_plus = np.where(
        (votes_2022_plus == 0) & (mvp_df_clean["VotePoints"] > 0),
        1,
        votes_2022_plus
    )

    mvp_df_clean["Votes"] = np.where(
        mvp_df_clean["Year"] >= 2022,
        votes_2022_plus,  # Convert points to ballot count for 2022+
        mvp_df_clean["VotePoints"]   # Use original vote count for pre-2022
    )

# Defensive stat duplicate handling
# Find all Solo columns and consolidate
solo_cols = [c for c in mvp_df_clean.columns if "solo" in c.lower()]
if solo_cols:
    for col in solo_cols:
        mvp_df_clean[col] = pd.to_numeric(mvp_df_clean[col], errors = "coerce")
    mvp_df_clean["Def_Solo"] = mvp_df_clean[solo_cols].bfill(axis = 1).iloc[:, 0]
    mvp_df_clean = mvp_df_clean.drop(columns = solo_cols)

# Find all Sack columns and consolidate
sack_cols = [c for c in mvp_df_clean.columns if "sk" in c.lower()]
if sack_cols:
    for col in sack_cols:
        mvp_df_clean[col] = pd.to_numeric(mvp_df_clean[col], errors = "coerce")
    mvp_df_clean["Def_Sk"] = mvp_df_clean[sack_cols].bfill(axis = 1).iloc[:, 0]
    mvp_df_clean = mvp_df_clean.drop(columns = sack_cols)

# Find all Interception columns and consolidate (exclude Pass_Int)
int_cols = [c for c in mvp_df_clean.columns if "int" in c.lower() and "pass" not in c.lower() and c != "Pass_Int"]
if int_cols:
    for col in int_cols:
        mvp_df_clean[col] = pd.to_numeric(mvp_df_clean[col], errors = "coerce")
    mvp_df_clean["Def_Int"] = mvp_df_clean[int_cols].bfill(axis = 1).iloc[:, 0]
    mvp_df_clean = mvp_df_clean.drop(columns = int_cols)

# Stat columns to numeric
numeric_columns = [
    "Rank", "VotePoints", "FirstPlaceVotes", "Votes",
    "G", "GS", "Year",
    "Pass_Cmp", "Pass_Att", "Pass_Yds", "Pass_TD", "Pass_Int",
    "Rush_Att", "Rush_Yds", "Rush_TD",
    "Rec_Rec", "Rec_Yds", "Rec_TD",
    "Def_Solo", "Def_Sk", "Def_Int"
]

for col in numeric_columns:
    if col in mvp_df_clean.columns:
        mvp_df_clean[col] = pd.to_numeric(mvp_df_clean[col], errors = "coerce")

# Clean text columns
for col in ["Position", "Player", "Team"]:
    if col in mvp_df_clean.columns:
        mvp_df_clean[col] = mvp_df_clean[col].astype(str).str.strip()

# Drop unwanted columns
cols_to_drop = ["VoteShare_1", "VoteShare_2", "VotePoints", "FirstPlaceVotes"]  # Keep only Votes
mvp_df_clean = mvp_df_clean.drop(columns = [c for c in cols_to_drop if c in mvp_df_clean.columns], errors = "ignore")

# Reorder
preferred_order = [
    "Year", "Rank", "Player", "Team", "Position", "Votes",
    "G", "GS",
    "Pass_Cmp", "Pass_Att", "Pass_Yds", "Pass_TD", "Pass_Int",
    "Rush_Att", "Rush_Yds", "Rush_TD",
    "Rec_Rec", "Rec_Yds", "Rec_TD",
    "Def_Solo", "Def_Sk", "Def_Int"
]

final_columns = [c for c in preferred_order if c in mvp_df_clean.columns]
final_columns.extend([c for c in mvp_df_clean.columns if c not in final_columns])
mvp_df_clean = mvp_df_clean[final_columns]



In [ ]:
mvp_df_clean.to_csv('mvp_df_clean.csv')
files.download('mvp_df_clean.csv')

# Extra Stats Merging

In [ ]:
# Code to skip the merging and cleaning
mvp_adv_clean_path = '/content/drive/MyDrive/CS3540/Project/mvp_adv_clean.csv'
mvp_adv_clean = pd.read_csv(mvp_adv_clean_path, index_col = 0)
mvp_adv_clean.head(50)

,Year,Rank,Player,Votes,Games,Games_Started,Age,Pass_QBrec,Pass_Rate,Pass_QBR,...,Scrim_Scrimmage_RRTD,Def_Solo,Def_Sk,Def_Int,Scrim_Rushing_1D,Scrim_Rushing_Succ_Pct,Scrim_Rushing_Lng,Scrim_Rushing_Y_A,Scrim_Rushing_Y_G,Scrim_Rushing_A_G
0,2024,1,Josh Allen,90.0,17,17,28.0,NaN,101.4,74.8,...,13.0,0.0,0.0,0.0,52.0,64.7,30.0,5.2,31.2,6.0
1,2024,2,Lamar Jackson,84.0,17,17,27.0,NaN,119.6,74.8,...,4.0,0.0,0.0,0.0,47.0,56.1,48.0,6.6,53.8,8.2
2,2024,3,Saquon Barkley,28.0,16,16,27.0,NaN,NaN,0.1,...,15.0,0.0,0.0,0.0,82.0,52.5,72.0,5.8,125.3,21.6
3,2024,4,Joe Burrow,20.0,17,17,28.0,NaN,108.5,72.1,...,2.0,0.0,0.0,0.0,16.0,47.6,47.0,4.8,11.8,2.5
4,2024,5,Jared Goff,11.0,17,17,30.0,NaN,111.8,65.4,...,1.0,0.0,0.0,0.0,9.0,34.3,10.0,1.6,3.3,2.1
5,2024,6,Patrick Mahomes,7.0,16,16,29.0,NaN,93.5,64.6,...,2.0,0.0,0.0,0.0,22.0,50.0,33.0,5.3,19.2,3.6
6,2024,7,Jayden Daniels,2.0,17,17,24.0,NaN,100.1,67.7,...,6.0,0.0,0.0,0.0,55.0,62.8,46.0,6.0,52.4,8.7
7,2024,8,Ja'Marr Chase,1.0,17,16,24.0,NaN,NaN,NaN,...,17.0,0.0,0.0,0.0,2.0,100.0,14.0,10.7,1.9,0.2
8,2024,9,Justin Herbert,1.0,17,17,26.0,NaN,101.7,61.7,...,2.0,0.0,0.0,0.0,23.0,46.4,41.0,4.4,18.0,4.1
9,2024,10,Sam Darnold,1.0,17,17,27.0,NaN,102.5,57.3,...,1.0,0.0,0.0,0.0,23.0,41.8,19.0,3.2,12.5,3.9


In [ ]:
def _flatten_cols(df: pd.DataFrame) -> pd.DataFrame:
    """Flatten MultiIndex columns and trim; make sure we have a clean 'Player' col name."""
    if isinstance(df.columns, pd.MultiIndex):
        df.columns = [' '.join(str(x).strip() for x in tup if str(x) != 'nan').strip()
                      for tup in df.columns]
    else:
        df.columns = [str(c).strip() for c in df.columns]

    # Normalize any column that looks like Player
    for c in list(df.columns):
        if re.fullmatch(r'(?i).*player.*', c):
            df = df.rename(columns={c: 'Player'})
            break
    return df

def _clean_player_name(s: str) -> str:
    """Strip PFR decorations like '*' '+' and whitespace."""
    if pd.isna(s):
        return s
    # Remove * and + and trailing team/year decorations if any
    s = str(s)
    s = s.replace('*', '').replace('+', '')
    s = re.sub(r'\s+', ' ', s).strip()
    return s

def _prefer_totals_rows(df: pd.DataFrame, team_col_candidates=('Tm','Team')):
    """If a totals row exists (e.g., 2TM/3TM/TOT) keep that; else keep the single-team row."""
    team_col = next((c for c in team_col_candidates if c in df.columns), None)
    if team_col is None or 'Player' not in df.columns:
        return df

    df = df.copy()
    # mark totals
    is_total = df[team_col].astype(str).str.contains(r'(?:\d+TM|TOT)', regex=True, na=False)
    # keep totals if they exist for a player, else keep the row with the max G or Yds as fallback
    totals = df[is_total]
    non_totals = df[~is_total]

    # players that have totals
    has_total = set(totals['Player'])

    # For players with totals -> keep only totals
    keep_totals = totals

    # For players without totals -> choose one row per player (max games, then max yards)
    candidates = non_totals[~non_totals['Player'].isin(has_total)].copy()

    if not candidates.empty:
        # create ranking keys
        if 'G' in candidates.columns:
            candidates['__rank_key1'] = pd.to_numeric(candidates['G'], errors='coerce').fillna(-1)
        else:
            candidates['__rank_key1'] = -1

        yds_col = next((c for c in candidates.columns if re.fullmatch(r'(?i).*\byds\b.*', c)), None)
        if yds_col:
            candidates['__rank_key2'] = pd.to_numeric(candidates[yds_col], errors='coerce').fillna(-1)
        else:
            candidates['__rank_key2'] = -1

        candidates = (candidates.sort_values(['Player','__rank_key1','__rank_key2'], ascending=[True, False, False])
                                 .drop_duplicates(subset=['Player'], keep='first')
                                 .drop(columns=['__rank_key1','__rank_key2'], errors='ignore'))

    out = pd.concat([keep_totals, candidates], ignore_index=True)
    return out

def _pick_main_table(tables: list) -> Optional[pd.DataFrame]:
    """Among read_html results, pick the most plausible stats table."""
    best = None
    best_cols = -1

    for i, t in enumerate(tables):
        t = _flatten_cols(t)
        cols = set(map(str, t.columns))

        # Look for Player column and some stats columns
        has_player = any(re.fullmatch(r'(?i).*player.*', c) for c in cols)
        has_stats = any('Yds' in c or 'TD' in c or 'Cmp' in c or 'Att' in c for c in cols)

        if has_player and has_stats and t.shape[1] > best_cols:
            best = t
            best_cols = t.shape[1]

    return best

def fetch_passing(year: int, max_retries: int = 3) -> Optional[pd.DataFrame]:
    """Fetch passing stats with retry logic and better error handling."""
    url = f'https://www.pro-football-reference.com/years/{year}/passing.htm'

    for attempt in range(max_retries):
        try:
            tables = pd.read_html(url)
            t = _pick_main_table(tables)
            if t is None:
                print(f"No suitable passing table found for {year}")
                return None

            t = _flatten_cols(t)

            # Debug: Show columns found
            print(f"  Passing columns for {year}: {t.columns.tolist()}")

            t['Year'] = year
            t['Player'] = t['Player'].apply(_clean_player_name)
            t = _prefer_totals_rows(t)

            # Keep focused subset - be more flexible with column names
            keep_cols = ['Year','Player']

            # Map common column variations to standardized names
            col_mapping = {
                'Team': 'Tm', 'Pos': 'Position', 'Pos.': 'Position'
            }

            # Apply mapping
            for old_name, new_name in col_mapping.items():
                if old_name in t.columns:
                    t = t.rename(columns={old_name: new_name})

            # Look for common passing columns with flexible matching
            potential_cols = ['Tm','Age','Pos','Position','G','GS','QBrec','Cmp','Att','Yds','TD','Int',
                             'Rate','Passer Rating','QBR','Y/A','AY/A','Y/C','Y/G','NY/A','ANY/A','Sk','4QC','GWD']

            passing_cols = []
            for col in potential_cols:
                if col in t.columns:
                    passing_cols.append(col)
                else:
                    # Look for similar column names
                    matches = [c for c in t.columns if col.lower() in c.lower() or c.lower() in col.lower()]
                    if matches:
                        passing_cols.append(matches[0])

            # Remove duplicates while preserving order
            passing_cols = list(dict.fromkeys(passing_cols))
            keep_cols.extend(passing_cols)

            # Filter to only columns that exist
            keep_cols = [c for c in keep_cols if c in t.columns]
            t = t[keep_cols].copy()

            # Add prefix to avoid conflicts (except Year, Player)
            rename_map = {}
            for c in passing_cols:
                if c not in ['Player', 'Year']:
                    rename_map[c] = f'PFR_Pass_{c}'

            t = t.rename(columns=rename_map)

            print(f"  Final passing columns: {t.columns.tolist()}")
            return t

        except Exception as e:
            print(f"Passing fetch attempt {attempt + 1}/{max_retries} failed for {year}: {e}")
            if attempt < max_retries - 1:
                time.sleep(2)
            else:
                return None

def fetch_scrimmage(year: int, max_retries: int = 3) -> Optional[pd.DataFrame]:
    """Fetch scrimmage stats with retry logic and better error handling."""
    url = f'https://www.pro-football-reference.com/years/{year}/scrimmage.htm'

    for attempt in range(max_retries):
        try:
            tables = pd.read_html(url)
            t = _pick_main_table(tables)
            if t is None:
                print(f"No suitable scrimmage table found for {year}")
                return None

            t = _flatten_cols(t)

            # Debug: Show columns found
            print(f"  Scrimmage columns for {year}: {t.columns.tolist()}")

            t['Year'] = year
            t['Player'] = t['Player'].apply(_clean_player_name)
            t = _prefer_totals_rows(t)

            keep_cols = ['Year','Player']

            # Map common column variations
            col_mapping = {
                'Team': 'Tm', 'Pos': 'Position', 'Pos.': 'Position'
            }

            # Apply mapping
            for old_name, new_name in col_mapping.items():
                if old_name in t.columns:
                    t = t.rename(columns={old_name: new_name})

            # Look for scrimmage columns with flexible matching
            potential_cols = ['Tm','Age','Pos','Position','G','GS',
                             'Rushing Att','Rushing Yds','Rushing TD','Rushing Y/A',
                             'Receiving Tgt','Receiving Rec','Receiving Yds','Receiving TD','Receiving Y/R',
                             'Scrimmage Yds','Scrimmage TD','All-Purpose Yds',
                             'Att','Yds','TD','Tgt','Rec','Y/R','Y/A']

            scr_cols = []
            for col in potential_cols:
                if col in t.columns:
                    scr_cols.append(col)
                else:
                    # Look for partial matches
                    matches = [c for c in t.columns if col.lower() in c.lower()]
                    if matches:
                        scr_cols.extend(matches)

            # Remove duplicates while preserving order
            scr_cols = list(dict.fromkeys(scr_cols))
            keep_cols.extend(scr_cols)

            # Filter to only columns that exist
            keep_cols = [c for c in keep_cols if c in t.columns]
            t = t[keep_cols].copy()

            # Add prefix to avoid conflicts
            rename_map = {}
            for c in scr_cols:
                if c not in ['Player', 'Year']:
                    # Clean up the column name for the prefix
                    clean_col = c.replace(' ', '_').replace('/', '_')
                    rename_map[c] = f'PFR_Scrim_{clean_col}'

            t = t.rename(columns=rename_map)

            print(f"  Final scrimmage columns: {t.columns.tolist()}")
            return t

        except Exception as e:
            print(f"Scrimmage fetch attempt {attempt + 1}/{max_retries} failed for {year}: {e}")
            if attempt < max_retries - 1:
                time.sleep(2)
            else:
                return None

def merge_nfl_stats(mvp_df_clean: pd.DataFrame, sleep_time: float = 2.0, debug: bool = True) -> pd.DataFrame:
    """
    Main function to merge MVP data with passing and scrimmage stats.

    Args:
        mvp_df_clean: Your cleaned MVP DataFrame
        sleep_time: Seconds to sleep between API calls
        debug: Whether to show detailed debugging info

    Returns:
        Merged DataFrame
    """
    # Validate input
    if 'Player' not in mvp_df_clean.columns or 'Year' not in mvp_df_clean.columns:
        raise ValueError("mvp_df_clean must contain 'Player' and 'Year' columns")

    # Prepare MVP data
    mvp_df_aug = mvp_df_clean.copy()
    mvp_df_aug['Player_clean'] = mvp_df_aug['Player'].apply(_clean_player_name)

    # Get unique years - focus on more recent years first for testing
    years = sorted(mvp_df_aug['Year'].dropna().astype(int).unique().tolist(), reverse=True)
    print(f"Processing years: {years}")

    if debug:
        print(f"MVP data shape: {mvp_df_aug.shape}")
        print(f"Sample MVP players for recent years:")
        recent_sample = mvp_df_aug[mvp_df_aug['Year'] >= 2020][['Year', 'Player', 'Player_clean']].head(10)
        print(recent_sample)

    pass_frames = []
    scrim_frames = []

    # Fetch data year by year
    for i, year in enumerate(years):
        print(f"\n{'='*50}")
        print(f"Processing year {year} ({i+1}/{len(years)})...")

        # Get MVP players for this year to check matches
        year_mvp_players = set(mvp_df_aug[mvp_df_aug['Year'] == year]['Player_clean'].dropna())
        print(f"MVP players in {year}: {sorted(year_mvp_players)}")

        # Fetch passing data
        passing_data = fetch_passing(year)
        if passing_data is not None:
            passing_data['Player_clean'] = passing_data['Player']

            if debug:
                # Show overlap
                pfr_players = set(passing_data['Player_clean'].dropna())
                overlap = year_mvp_players.intersection(pfr_players)
                print(f"  Passing overlap: {len(overlap)}/{len(year_mvp_players)} MVP players found")
                if overlap:
                    print(f"    Matched: {sorted(overlap)}")
                missing = year_mvp_players - pfr_players
                if missing:
                    print(f"    Missing: {sorted(missing)}")

            pass_frames.append(passing_data)
            print(f"  Passing data: {passing_data.shape[0]} players, {passing_data.shape[1]} columns")
        else:
            print(f"  No passing data for {year}")

        time.sleep(sleep_time)

        # Fetch scrimmage data
        scrimmage_data = fetch_scrimmage(year)
        if scrimmage_data is not None:
            scrimmage_data['Player_clean'] = scrimmage_data['Player']

            if debug:
                # Show overlap
                pfr_players = set(scrimmage_data['Player_clean'].dropna())
                overlap = year_mvp_players.intersection(pfr_players)
                print(f"  Scrimmage overlap: {len(overlap)}/{len(year_mvp_players)} MVP players found")
                if overlap:
                    print(f"    Matched: {sorted(overlap)}")
                missing = year_mvp_players - pfr_players
                if missing:
                    print(f"    Missing: {sorted(missing)}")

            scrim_frames.append(scrimmage_data)
            print(f"  Scrimmage data: {scrimmage_data.shape[0]} players, {scrimmage_data.shape[1]} columns")
        else:
            print(f"  No scrimmage data for {year}")

        time.sleep(sleep_time)

        # Stop after a few years for testing
        if debug and i >= 2:  # Process only 3 years for initial testing
            print(f"\nStopping after {i+1} years for testing...")
            break

    # Combine all data
    pass_all = pd.concat(pass_frames, ignore_index=True) if pass_frames else pd.DataFrame()
    scrim_all = pd.concat(scrim_frames, ignore_index=True) if scrim_frames else pd.DataFrame()

    print(f"\n{'='*50}")
    print(f"SUMMARY:")
    print(f"Combined passing data: {pass_all.shape}")
    print(f"Combined scrimmage data: {scrim_all.shape}")

    # Merge with MVP data (left joins to preserve all MVP entries)
    merged = mvp_df_aug.copy()

    if not pass_all.empty:
        print(f"\nMerging passing data...")
        before_cols = merged.shape[1]
        merged = merged.merge(
            pass_all.drop(columns=['Player'], errors='ignore'),
            on=['Year','Player_clean'],
            how='left',
            suffixes=('', '_pass_dup')
        )
        added_cols = merged.shape[1] - before_cols
        print(f"Added {added_cols} passing columns")

    if not scrim_all.empty:
        print(f"Merging scrimmage data...")
        before_cols = merged.shape[1]
        merged = merged.merge(
            scrim_all.drop(columns=['Player'], errors='ignore'),
            on=['Year','Player_clean'],
            how='left',
            suffixes=('', '_scrim_dup')
        )
        added_cols = merged.shape[1] - before_cols
        print(f"Added {added_cols} scrimmage columns")

    # Clean up
    merged = merged.drop(columns=['Player_clean'])

    # Convert numeric columns
    numeric_cols = []
    for col in merged.columns:
        if any(tag in col for tag in ['PFR_Pass_','PFR_Scrim_']):
            merged[col] = pd.to_numeric(merged[col], errors='ignore')
            numeric_cols.append(col)

    print(f"\nFinal merged shape: {merged.shape}")
    print(f"Added numeric columns: {len(numeric_cols)}")

    # Show merge success rate
    if not pass_all.empty or not scrim_all.empty:
        total_mvp = len(mvp_df_aug)
        # Count rows with any PFR data
        has_pfr_data = merged.dropna(subset=[col for col in merged.columns if 'PFR_' in col], how='all')
        successful_merges = len(has_pfr_data)
        print(f"Successfully merged stats for {successful_merges}/{total_mvp} MVP entries ({successful_merges/total_mvp*100:.1f}%)")

        if debug and successful_merges > 0:
            print(f"\nSample of merged data:")
            sample_cols = ['Year', 'Player', 'Team', 'Position'] + [c for c in merged.columns if 'PFR_' in c][:5]
            sample_cols = [c for c in sample_cols if c in merged.columns]
            print(has_pfr_data[sample_cols].head())

    return merged

mvp_advanced = merge_nfl_stats(mvp_df_clean, sleep_time = 3, debug = False)

In [ ]:
mvp_advanced.to_csv('mvp_advanced.csv')
files.download('mvp_advanced.csv')

In [ ]:
def clean_merged_nfl_data(df: pd.DataFrame, prefer_pfr: bool = True) -> pd.DataFrame:
    """
    Clean and consolidate the merged NFL MVP + PFR data.
    Removes duplicates, per-game stats, and creates a streamlined dataset.

    Args:
        df: The merged DataFrame with MVP and PFR columns
        prefer_pfr: If True, prefer PFR data over MVP data for stats when both exist

    Returns:
        Cleaned DataFrame with no duplicates or per-game stats
    """

    print("Starting comprehensive data cleaning...")
    print(f"Original shape: {df.shape}")
    print(f"Original columns: {len(df.columns)}")

    # Make a copy to work with
    cleaned = df.copy()

    # Step 1: Remove obvious unwanted columns
    print("\n" + "="*50)
    print("STEP 1: Removing unwanted columns")

    unwanted_cols = []

    # Remove unnamed index columns
    for col in cleaned.columns:
        if col.startswith('Unnamed:') and ('level_0' in col or col == 'Unnamed: 0'):
            unwanted_cols.append(col)

    if unwanted_cols:
        cleaned = cleaned.drop(columns=unwanted_cols)
        print(f"Removed {len(unwanted_cols)} unnamed index columns")

    # Step 2: Fix messy column names
    print("\nSTEP 2: Fixing messy column names")

    name_fixes = {}
    for col in cleaned.columns:
        if 'Unnamed:' in col and 'level_0' in col:
            if 'Age' in col:
                name_fixes[col] = 'PFR_Scrim_Age'
            elif 'Pos' in col:
                name_fixes[col] = 'PFR_Scrim_Position'
            elif col.endswith('G') and 'GS' not in col:
                name_fixes[col] = 'PFR_Scrim_G'
            elif 'GS' in col:
                name_fixes[col] = 'PFR_Scrim_GS'

    if name_fixes:
        cleaned = cleaned.rename(columns=name_fixes)
        print(f"Fixed {len(name_fixes)} messy column names")

    # Step 3: Identify and remove per-game stats
    print("\nSTEP 3: Removing per-game statistics")

    per_game_indicators = ['/G', 'per G', '_G_', 'Per_Game', 'Y_G', 'R_G', 'A_G']
    per_game_cols = []

    for col in cleaned.columns:
        if any(indicator in col for indicator in per_game_indicators):
            per_game_cols.append(col)

    if per_game_cols:
        cleaned = cleaned.drop(columns=per_game_cols)
        print(f"Removed {len(per_game_cols)} per-game stat columns")
        print(f"Examples: {per_game_cols[:5]}")

    # Step 4: Consolidate duplicate stat columns
    print("\nSTEP 4: Consolidating duplicate statistics")

    # Define stat mappings - what should be consolidated
    stat_mappings = {
        'Games': ['G', 'PFR_Pass_G', 'PFR_Scrim_G'],
        'Games_Started': ['GS', 'PFR_Pass_GS', 'PFR_Scrim_GS'],
        'Age': ['PFR_Pass_Age', 'PFR_Scrim_Age'],
        'Team': ['Team', 'PFR_Pass_Tm'],
        'Position': ['Position', 'PFR_Pass_Position', 'PFR_Scrim_Position'],

        # Passing stats
        'Pass_Completions': ['Pass_Cmp', 'PFR_Pass_Cmp'],
        'Pass_Attempts': ['Pass_Att', 'PFR_Pass_Att'],
        'Pass_Yards': ['Pass_Yds', 'PFR_Pass_Yds'],
        'Pass_Touchdowns': ['Pass_TD', 'PFR_Pass_TD'],
        'Pass_Interceptions': ['Pass_Int', 'PFR_Pass_Int'],

        # Rushing stats
        'Rush_Attempts': ['Rush_Att', 'PFR_Scrim_Rushing_Att'],
        'Rush_Yards': ['Rush_Yds', 'PFR_Scrim_Rushing_Yds'],
        'Rush_Touchdowns': ['Rush_TD', 'PFR_Scrim_Rushing_TD'],

        # Receiving stats
        'Rec_Receptions': ['Rec_Rec', 'PFR_Scrim_Receiving_Rec'],
        'Rec_Yards': ['Rec_Yds', 'PFR_Scrim_Receiving_Yds'],
        'Rec_Touchdowns': ['Rec_TD', 'PFR_Scrim_Receiving_TD'],
    }

    consolidated_count = 0
    dropped_count = 0

    for final_name, source_cols in stat_mappings.items():
        available_cols = [col for col in source_cols if col in cleaned.columns]

        if len(available_cols) > 1:  # Only consolidate if we have multiple columns
            # Determine priority order
            if prefer_pfr:
                pfr_cols = [col for col in available_cols if 'PFR_' in col]
                mvp_cols = [col for col in available_cols if 'PFR_' not in col]
                priority_order = pfr_cols + mvp_cols
            else:
                mvp_cols = [col for col in available_cols if 'PFR_' not in col]
                pfr_cols = [col for col in available_cols if 'PFR_' in col]
                priority_order = mvp_cols + pfr_cols

            # Create consolidated column
            cleaned[final_name] = pd.Series(dtype='object', index=cleaned.index)

            # Fill from highest to lowest priority
            for col in reversed(priority_order):
                mask = cleaned[final_name].isna() & cleaned[col].notna()
                cleaned.loc[mask, final_name] = cleaned.loc[mask, col]

            # Drop the source columns
            cleaned = cleaned.drop(columns=available_cols)
            consolidated_count += 1
            dropped_count += len(available_cols)

        elif len(available_cols) == 1:
            # Just rename single column if needed
            old_name = available_cols[0]
            if old_name != final_name:
                cleaned = cleaned.rename(columns={old_name: final_name})

    print(f"Consolidated {consolidated_count} stat types")
    print(f"Dropped {dropped_count} redundant columns")

    # Step 5: Keep useful PFR advanced stats (no per-game stats)
    print("\nSTEP 5: Filtering PFR advanced statistics")

    # Define what advanced stats to keep
    keep_indicators = [
        'Rate', 'QBR', 'Y/A', 'AY/A', 'Y/C', 'NY/A', 'ANY/A',  # Passing efficiency
        'Sk', '4QC', 'GWD', 'QBrec',  # Passing situational
        'Tgt', 'Succ%', 'Lng', 'Ctch%', 'Y_R', 'Y_Tgt',  # Receiving efficiency
        'Touch', 'Y_Tch', '1D'  # General efficiency
    ]

    kept_advanced = []
    removed_advanced = []

    for col in list(cleaned.columns):
        if 'PFR_' in col:
            # Check if it's an advanced stat we want to keep
            if any(indicator in col for indicator in keep_indicators):
                kept_advanced.append(col)
            else:
                # Remove PFR columns that are just basic stats we already consolidated
                cleaned = cleaned.drop(columns=[col])
                removed_advanced.append(col)

    print(f"Kept {len(kept_advanced)} useful PFR advanced stats")
    print(f"Removed {len(removed_advanced)} redundant PFR columns")

    # Step 6: Clean up remaining PFR column names
    print("\nSTEP 6: Cleaning PFR column names")

    name_cleanup = {}
    for col in cleaned.columns:
        if 'PFR_' in col:
            # Simplify the names
            new_name = col.replace('PFR_Pass_', '').replace('PFR_Scrim_', '')
            new_name = new_name.replace('_', ' ').replace('/', ' per ').replace('%', ' Pct')
            new_name = re.sub(r'\s+', '_', new_name).strip('_')

            # Add prefixes for clarity
            if any(pass_stat in col for pass_stat in ['Pass', 'Rate', 'QBR', 'Sk', 'QBrec', '4QC', 'GWD']):
                new_name = f"Pass_{new_name}"
            elif 'Receiving' in col or any(rec_stat in col for rec_stat in ['Tgt', 'Ctch', 'Rec']):
                new_name = f"Rec_{new_name}"
            elif 'Rushing' in col:
                new_name = f"Rush_{new_name}"
            elif 'Scrimmage' in col:
                new_name = f"Scrim_{new_name}"

            name_cleanup[col] = new_name

    if name_cleanup:
        cleaned = cleaned.rename(columns=name_cleanup)
        print(f"Cleaned {len(name_cleanup)} PFR column names")

    # Step 7: Final duplicate detection and removal
    print("\nSTEP 7: Final duplicate detection")

    # Check for columns with identical or nearly identical data
    duplicate_pairs = []
    cols_list = list(cleaned.columns)

    for i, col1 in enumerate(cols_list):
        for col2 in cols_list[i+1:]:
            if col1 != col2:
                # Compare the actual data (handle NaNs)
                series1 = cleaned[col1].fillna(-999999)  # Use unlikely value for NaN
                series2 = cleaned[col2].fillna(-999999)

                if series1.equals(series2):
                    duplicate_pairs.append((col1, col2))

    # Remove duplicates, keeping the better column name
    final_removals = []
    for col1, col2 in duplicate_pairs:
        # Keep the shorter, cleaner name
        if len(col1) <= len(col2) and 'PFR_' not in col1:
            final_removals.append(col2)
        elif 'PFR_' not in col2:
            final_removals.append(col1)
        else:
            final_removals.append(col1 if len(col1) > len(col2) else col2)

    if final_removals:
        cleaned = cleaned.drop(columns=final_removals)
        print(f"Removed {len(final_removals)} final duplicate columns")

    # Step 8: Convert data types
    print("\nSTEP 8: Converting data types")

    # Convert numeric columns
    numeric_cols = []
    text_cols = ['Player', 'Team', 'Position']

    for col in cleaned.columns:
        if col not in text_cols and col != 'Year':
            original_type = cleaned[col].dtype
            cleaned[col] = pd.to_numeric(cleaned[col], errors='coerce')
            if str(original_type) != str(cleaned[col].dtype):
                numeric_cols.append(col)

    # Clean text columns
    for col in text_cols:
        if col in cleaned.columns:
            cleaned[col] = cleaned[col].astype(str).str.strip()

    print(f"Converted {len(numeric_cols)} columns to numeric")

    # Step 9: Organize columns logically
    print("\nSTEP 9: Organizing columns")

    # Define column groups
    base_info = ['Year', 'Rank', 'Player', 'Team', 'Position', 'Votes']
    game_info = ['Games', 'Games_Started', 'Age']

    passing_stats = [col for col in cleaned.columns if col.startswith('Pass_')]
    rushing_stats = [col for col in cleaned.columns if col.startswith('Rush_')]
    receiving_stats = [col for col in cleaned.columns if col.startswith('Rec_')]
    scrimmage_stats = [col for col in cleaned.columns if col.startswith('Scrim_')]
    defense_stats = [col for col in cleaned.columns if col.startswith('Def_')]

    # Remaining columns
    accounted_for = set(base_info + game_info + passing_stats + rushing_stats +
                       receiving_stats + scrimmage_stats + defense_stats)
    other_cols = [col for col in cleaned.columns if col not in accounted_for]

    # Build final column order
    final_order = []
    for group in [base_info, game_info, passing_stats, rushing_stats,
                  receiving_stats, scrimmage_stats, defense_stats, other_cols]:
        final_order.extend([col for col in group if col in cleaned.columns])

    cleaned = cleaned[final_order]

    # Final summary
    print("\n" + "="*50)
    print("CLEANING COMPLETE!")
    print(f"Final shape: {cleaned.shape}")
    print(f"Columns removed: {df.shape[1] - cleaned.shape[1]}")
    print(f"Final columns: {len(cleaned.columns)}")

    return cleaned

def get_data_summary(df: pd.DataFrame) -> pd.DataFrame:
    """
    Get a comprehensive summary of the cleaned data.
    """
    summary_data = []

    for col in df.columns:
        non_null = df[col].notna().sum()
        null_count = df[col].isna().sum()
        completeness = (non_null / len(df)) * 100

        # Get sample values
        sample_vals = df[col].dropna()
        if len(sample_vals) > 0:
            sample = sample_vals.iloc[:3].tolist()
            sample_str = str(sample)
        else:
            sample_str = 'No data'

        summary_data.append({
            'Column': col,
            'Non_Null_Count': non_null,
            'Null_Count': null_count,
            'Completeness_Pct': round(completeness, 1),
            'Data_Type': str(df[col].dtype),
            'Sample_Values': sample_str
        })

    summary = pd.DataFrame(summary_data)
    summary = summary.sort_values('Completeness_Pct', ascending=False)

    return summary

def analyze_data_quality(df: pd.DataFrame) -> None:
    """
    Print a comprehensive analysis of the data quality.
    """
    print("DATA QUALITY ANALYSIS")
    print("="*50)

    total_rows = len(df)
    total_cols = len(df.columns)

    print(f"Dataset size: {total_rows:,} rows × {total_cols} columns")

    # Completeness by category
    categories = {
        'Base Info': [c for c in df.columns if c in ['Year', 'Rank', 'Player', 'Team', 'Position', 'Votes']],
        'Passing': [c for c in df.columns if c.startswith('Pass_')],
        'Rushing': [c for c in df.columns if c.startswith('Rush_')],
        'Receiving': [c for c in df.columns if c.startswith('Rec_')],
        'Defense': [c for c in df.columns if c.startswith('Def_')]
    }

    print(f"\nCompleteness by category:")
    for cat_name, cat_cols in categories.items():
        if cat_cols:
            avg_completeness = df[cat_cols].notna().mean().mean() * 100
            print(f"  {cat_name}: {avg_completeness:.1f}% ({len(cat_cols)} columns)")

    # Years covered
    if 'Year' in df.columns:
        years = df['Year'].dropna()
        print(f"\nYears covered: {int(years.min())} - {int(years.max())} ({len(years.unique())} unique years)")

    # Player coverage
    if 'Player' in df.columns:
        unique_players = df['Player'].nunique()
        print(f"Unique players: {unique_players:,}")

In [ ]:
mvp_adv_clean = clean_merged_nfl_data(mvp_advanced, prefer_pfr = True)
summary = get_data_summary(mvp_adv_clean)
print(summary.to_string())

NameError: name 'clean_merged_nfl_data' is not defined

In [ ]:
mvp_adv_clean.to_csv('mvp_adv_clean.csv')
files.download('mvp_adv_clean.csv')

## Core info

Year — Season year.

Rank — Player’s finish on the MVP ballot (the table “Rk”) for that season.

Player — Player name.

Votes — Total MVP vote points (weighted by ballot, not just 1st-place votes).

Availability / age

Games — Games played (G).

Games_Started — Games started (GS).

Age — Player age as of Dec 31 of that season.

## Passing (QB-focused, others may be blank)

Pass_QBrec — Team record in games the QB started (W–L–T).

Pass_Rate — NFL Passer Rating.

Pass_QBR — ESPN Total QBR (0–100).

Pass_Y_per_A — Yards per attempt (Y/A) = Pass Yards ÷ Pass Attempts.

Pass_AY_per_A — Adjusted yards per attempt (AY/A)
Formula: (Pass Yards + 20×Pass TD − 45×INT) ÷ Pass Attempts.

Pass_Y_per_C — Yards per completion (Y/C) = Pass Yards ÷ Completions.

Pass_NY_per_A — Net yards per attempt (NY/A)
Formula: (Pass Yards − Sack Yards) ÷ (Pass Attempts + Sacks).

Pass_ANY_per_A — Adjusted net yards per attempt (ANY/A)
Formula: (Pass Yards − Sack Yards + 20×Pass TD − 45×INT) ÷ (Pass Attempts + Sacks).

Pass_Sk — Times sacked.

Pass_4QC — Fourth-quarter comebacks led by the QB.

Pass_GWD — Game-winning drives led by the QB.

Pass_Completions — Completed passes (Cmp).

Pass_Attempts — Pass attempts (Att).

Pass_Yards — Passing yards (Yds).

Pass_Touchdowns — Passing TDs (TD).

Pass_Interceptions — Interceptions thrown (Int).

## Rushing

Rush_Rushing_1D — First downs rushing (1D).

Rush_Rushing_Succ_Pct — Rushing Success Rate (Succ%)
A rush is “successful” if it gains ≥40% of yards to go on 1st down, ≥60% on 2nd, and 100% on 3rd/4th. Denominator = rushing attempts.

Rush_Rushing_Lng — Longest rush (Lng).

Rush_Attempts — Rushing attempts (Att).

Rush_Yards — Rushing yards (Yds).

Rush_Touchdowns — Rushing TDs (TD).

## Receiving

Rec_Receiving_Tgt — Targets (Tgt).

Rec_Receiving_Y_R — Yards per reception (Y/R) = Rec Yards ÷ Receptions.

Rec_Receiving_1D — First downs receiving (1D).

Rec_Receiving_Succ_Pct — Receiving Success Rate (Succ%)
Same 40/60/100% rule; denominator = targets.

Rec_Receiving_Lng — Longest reception (Lng).

Rec_Receiving_Ctch_Pct — Catch % (Ctch%) = Receptions ÷ Targets.

Rec_Receiving_Y_Tgt — Yards per target (Y/Tgt) = Rec Yards ÷ Targets.

Rec_Receptions — Receptions (Rec).

Rec_Yards — Receiving yards (Yds).

Rec_Touchdowns — Receiving TDs (TD).

## Scrimmage (rush + receive)

Scrim_Scrimmage_Touch — Touches = Rush Attempts + Receptions.

Scrim_Scrimmage_Y_Tch — Yards per touch (Y/Tch) = (Rush Yds + Rec Yds) ÷ Touches.

## Defense (often 0 for offensive players)

Def_Solo — Solo tackles credited.

Def_Sk — Sacks credited.

Def_Int — Interceptions made on defense.

## Notes & formulas (quick reference)

Rushing/Receiving Success Rate:
1st down: ≥40% of yards to go; 2nd: ≥60%; 3rd/4th: 100%.
Receiving denominator = targets; rushing denominator = rush attempts.

AY/A: (PassYds + 20×TD − 45×INT) / Att.

NY/A: (PassYds − SackYds) / (Att + Sacks).

ANY/A: (PassYds − SackYds + 20×TD − 45×INT) / (Att + Sacks).

QBR: ESPN’s play-by-play based QB value metric (0–100).

Passer Rating: NFL formula based on Cmp%, Y/A, TD%, INT%.

# OPOY Data

In [ ]:
# Skip scraping
opoy_df = '/content/drive/MyDrive/CS3540/Project/opoy_df.csv'
opoy_df = pd.read_csv(opoy_df, index_col = 0)
opoy_df.head(50)

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/CS3540/Project/opoy_df.csv'

In [ ]:
# Scrape OPOY (Offensive Player of the Year) data
year = 2024
opoy_dfs = []

while year >= 1980:
    url = 'https://www.pro-football-reference.com/awards/awards_' + str(year) + '.htm'

    try:
        # Get all tables from the page
        tables = pd.read_html(url)

        # tables[1] is OPOY voting
        opoy_table = tables[1]

        # Check if columns are multi-level and handle accordingly
        if isinstance(opoy_table.columns, pd.MultiIndex):
            # For multi-level columns, add year as a regular column after flattening
            opoy_table.columns = [' '.join(str(col).strip() for col in multi_col if str(col) != 'nan')
                                for multi_col in opoy_table.columns]
            opoy_table['Year'] = year
        else:
            # For regular columns, add year normally
            opoy_table['Year'] = year

        opoy_dfs.append(opoy_table)
        print(f"Scraped OPOY {year} - Shape: {opoy_table.shape}")

    except Exception as e:
        print(f"Failed to scrape OPOY {year}: {e}")

    time.sleep(3)
    year -= 1

# Combine all dataframes
opoy_df = pd.concat(opoy_dfs, ignore_index=True)
print(f"\nOPOY Total rows: {len(opoy_df)}")

# Check if Year column exists and show some info
print("\nColumns in OPOY dataframe:")
print(opoy_df.columns.tolist())

if 'Year' in opoy_df.columns:
    print(f"\nYear column added successfully")
    print(f"Years covered: {opoy_df['Year'].min()} - {opoy_df['Year'].max()}")
    print(f"Rows per year: {opoy_df['Year'].value_counts().sort_index()}")
else:
    print("Year column not found")
    print("Sample of dataframe:")
    print(opoy_df.head())

# Show sample of OPOY data
print("\nSample OPOY data:")
print(opoy_df.head(10))

In [ ]:
# Uncleaned OPOY
opoy_df.to_csv('opoy_df.csv')
files.download('opoy_df.csv')

In [ ]:
# Skip Cleaning
opoy_df_adv = '/content/drive/MyDrive/CS3540/Project/opoy_df_adv.csv'
opoy_df_adv = pd.read_csv(opoy_df_adv, index_col = 0)
opoy_df_adv.head(50)

,Year,Rank,Player,Team,Position,Votes,G,GS,Pass_Cmp,Pass_Att,...,PFR_Scrim_Receiving_Y_Tgt,PFR_Scrim_Rushing_Att,PFR_Scrim_Rushing_Yds,PFR_Scrim_Rushing_TD,PFR_Scrim_Rushing_1D,PFR_Scrim_Rushing_Succ%,PFR_Scrim_Rushing_Lng,PFR_Scrim_Rushing_Y_A,PFR_Scrim_Rushing_Y_G,PFR_Scrim_Rushing_A_G
0,2024,1,Saquon Barkley,Philadelphia Eagles,RB,94.0,16,16,0,0,...,6.5,345.0,2005.0,13.0,82.0,52.5,72.0,5.8,125.3,21.6
1,2024,2,Lamar Jackson,Baltimore Ravens,QB,43.0,17,17,316,474,...,NaN,139.0,915.0,4.0,47.0,56.1,48.0,6.6,53.8,8.2
2,2024,3,Ja'Marr Chase,Cincinnati Bengals,WR,40.0,17,16,0,0,...,9.8,3.0,32.0,0.0,2.0,100.0,14.0,10.7,1.9,0.2
3,2024,4,Derrick Henry,Baltimore Ravens,RB,21.0,17,17,0,0,...,8.8,325.0,1921.0,16.0,94.0,58.8,87.0,5.9,113.0,19.1
4,2024,5,Joe Burrow,Cincinnati Bengals,QB,19.0,17,17,460,652,...,NaN,42.0,201.0,2.0,16.0,47.6,47.0,4.8,11.8,2.5
5,2024,6,Josh Allen,Buffalo Bills,QB,11.0,17,17,307,483,...,NaN,102.0,531.0,12.0,52.0,64.7,30.0,5.2,31.2,6.0
6,2024,7,Justin Jefferson,Minnesota Vikings,WR,6.0,17,17,1,1,...,10.0,1.0,3.0,0.0,0.0,0.0,3.0,3.0,0.2,0.1
7,2024,8,Jahmyr Gibbs,Detroit Lions,RB,5.0,17,4,0,0,...,8.2,250.0,1412.0,16.0,70.0,53.6,70.0,5.6,83.1,14.7
8,2024,9,Jared Goff,Detroit Lions,QB,1.0,17,17,390,539,...,7.0,35.0,56.0,0.0,9.0,34.3,10.0,1.6,3.3,2.1
9,2024,10,Joe Thuney,Kansas City Chiefs,G,1.0,17,17,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
def clean_opoy_data(opoy_df):
    """
    Clean the scraped OPOY DataFrame similar to MVP cleaning.
    """
    print("Starting OPOY data cleaning...")
    print(f"Original shape: {opoy_df.shape}")

    opoy_df_clean = opoy_df.copy()

    # Drop unnecessary columns
    opoy_df_clean = opoy_df_clean.drop(columns=["Unnamed: 0"], errors="ignore")

    # Rename columns - adjust these based on actual column names
    column_mapping = {
        "Unnamed: 0_level_0 Rk": "Rank",
        "Unnamed: 1_level_0 Pos": "Position",
        "Unnamed: 2_level_0 Player": "Player",
        "Unnamed: 3_level_0 Tm": "Team",
        "Unnamed: 4_level_0 Vote Pts": "VotePoints_1",
        "Unnamed: 4_level_0 Votes": "VotePoints_2",
        "Unnamed: 5_level_0 1st Place": "FirstPlaceVotes",
        "Unnamed: 6_level_0 Share": "VoteShare_1",
        "Unnamed: 5_level_0 Share": "VoteShare_2",
        "Games G": "G",
        "Games GS": "GS",
        "Passing Cmp": "Pass_Cmp",
        "Passing Att": "Pass_Att",
        "Passing Yds": "Pass_Yds",
        "Passing TD": "Pass_TD",
        "Passing Int": "Pass_Int",
        "Rushing Att": "Rush_Att",
        "Rushing Yds": "Rush_Yds",
        "Rushing TD": "Rush_TD",
        "Receiving Rec": "Rec_Rec",
        "Receiving Yds": "Rec_Yds",
        "Receiving TD": "Rec_TD"
    }
    opoy_df_clean = opoy_df_clean.rename(columns=column_mapping)

    # Consolidate voting columns
    vote_points_cols = [c for c in ["VotePoints_1", "VotePoints_2"] if c in opoy_df_clean.columns]
    if vote_points_cols:
        opoy_df_clean["VotePoints"] = pd.to_numeric(opoy_df_clean[vote_points_cols[0]], errors="coerce")
        for col in vote_points_cols[1:]:
            opoy_df_clean["VotePoints"] = opoy_df_clean["VotePoints"].fillna(
                pd.to_numeric(opoy_df_clean[col], errors="coerce")
            )
        opoy_df_clean = opoy_df_clean.drop(columns=vote_points_cols)

    # Convert FirstPlaceVotes to numeric
    if "FirstPlaceVotes" in opoy_df_clean.columns:
        opoy_df_clean["FirstPlaceVotes"] = pd.to_numeric(opoy_df_clean["FirstPlaceVotes"], errors="coerce")

    # Create unified Votes column
    if "VotePoints" in opoy_df_clean.columns and "Year" in opoy_df_clean.columns:
        # 2022 and after: convert point totals to estimated ballot counts
        votes_2022_plus = opoy_df_clean["VotePoints"] / 4.2
        votes_2022_plus = np.where(pd.isna(votes_2022_plus), np.nan, np.round(votes_2022_plus))
        # Ensure anyone with points gets at least 1 vote
        votes_2022_plus = np.where(
            (votes_2022_plus == 0) & (opoy_df_clean["VotePoints"] > 0),
            1,
            votes_2022_plus
        )

        opoy_df_clean["Votes"] = np.where(
            opoy_df_clean["Year"] >= 2022,
            votes_2022_plus,
            opoy_df_clean["VotePoints"]
        )

    # Defensive stat duplicate handling (if any OPOY has defensive stats)
    # Find all Solo columns and consolidate
    solo_cols = [c for c in opoy_df_clean.columns if "solo" in c.lower()]
    if solo_cols:
        for col in solo_cols:
            opoy_df_clean[col] = pd.to_numeric(opoy_df_clean[col], errors="coerce")
        opoy_df_clean["Def_Solo"] = opoy_df_clean[solo_cols].bfill(axis=1).iloc[:, 0]
        opoy_df_clean = opoy_df_clean.drop(columns=solo_cols)

    # Find all Sack columns and consolidate
    sack_cols = [c for c in opoy_df_clean.columns if "sk" in c.lower()]
    if sack_cols:
        for col in sack_cols:
            opoy_df_clean[col] = pd.to_numeric(opoy_df_clean[col], errors="coerce")
        opoy_df_clean["Def_Sk"] = opoy_df_clean[sack_cols].bfill(axis=1).iloc[:, 0]
        opoy_df_clean = opoy_df_clean.drop(columns=sack_cols)

    # Find all Interception columns and consolidate (exclude Pass_Int)
    int_cols = [c for c in opoy_df_clean.columns if "int" in c.lower() and "pass" not in c.lower() and c != "Pass_Int"]
    if int_cols:
        for col in int_cols:
            opoy_df_clean[col] = pd.to_numeric(opoy_df_clean[col], errors="coerce")
        opoy_df_clean["Def_Int"] = opoy_df_clean[int_cols].bfill(axis=1).iloc[:, 0]
        opoy_df_clean = opoy_df_clean.drop(columns=int_cols)

    # Convert stat columns to numeric
    numeric_columns = [
        "Rank", "VotePoints", "FirstPlaceVotes", "Votes",
        "G", "GS", "Year",
        "Pass_Cmp", "Pass_Att", "Pass_Yds", "Pass_TD", "Pass_Int",
        "Rush_Att", "Rush_Yds", "Rush_TD",
        "Rec_Rec", "Rec_Yds", "Rec_TD",
        "Def_Solo", "Def_Sk", "Def_Int"
    ]

    for col in numeric_columns:
        if col in opoy_df_clean.columns:
            opoy_df_clean[col] = pd.to_numeric(opoy_df_clean[col], errors="coerce")

    # Clean text columns
    for col in ["Position", "Player", "Team"]:
        if col in opoy_df_clean.columns:
            opoy_df_clean[col] = opoy_df_clean[col].astype(str).str.strip()

    # Drop unwanted columns
    cols_to_drop = ["VoteShare_1", "VoteShare_2", "VotePoints", "FirstPlaceVotes"]
    opoy_df_clean = opoy_df_clean.drop(
        columns=[c for c in cols_to_drop if c in opoy_df_clean.columns],
        errors="ignore"
    )

    # Reorder columns
    preferred_order = [
        "Year", "Rank", "Player", "Team", "Position", "Votes",
        "G", "GS",
        "Pass_Cmp", "Pass_Att", "Pass_Yds", "Pass_TD", "Pass_Int",
        "Rush_Att", "Rush_Yds", "Rush_TD",
        "Rec_Rec", "Rec_Yds", "Rec_TD",
        "Def_Solo", "Def_Sk", "Def_Int"
    ]

    final_columns = [c for c in preferred_order if c in opoy_df_clean.columns]
    final_columns.extend([c for c in opoy_df_clean.columns if c not in final_columns])
    opoy_df_clean = opoy_df_clean[final_columns]

    print(f"Cleaned shape: {opoy_df_clean.shape}")
    print(f"Columns: {opoy_df_clean.columns.tolist()}")

    return opoy_df_clean

In [ ]:
# Merging OPOY with PFR stats

def merge_opoy_with_pfr(opoy_df_clean: pd.DataFrame, sleep_time: float = 2.0, debug: bool = True) -> pd.DataFrame:
    """
    Merge OPOY data with passing and scrimmage stats from Pro Football Reference.
    """
    if 'Player' not in opoy_df_clean.columns or 'Year' not in opoy_df_clean.columns:
        raise ValueError("opoy_df_clean must contain 'Player' and 'Year' columns")

    opoy_df_aug = opoy_df_clean.copy()
    opoy_df_aug['Player_clean'] = opoy_df_aug['Player'].apply(_clean_player_name)

    years = sorted(opoy_df_aug['Year'].dropna().astype(int).unique().tolist(), reverse=True)
    print(f"Processing years: {years}")

    if debug:
        print(f"OPOY data shape: {opoy_df_aug.shape}")
        print(f"Sample OPOY players for recent years:")
        recent_sample = opoy_df_aug[opoy_df_aug['Year'] >= 2020][['Year', 'Player', 'Player_clean']].head(10)
        print(recent_sample)

    pass_frames = []
    scrim_frames = []

    for i, year in enumerate(years):
        print(f"\n{'='*50}")
        print(f"Processing year {year} ({i+1}/{len(years)})...")

        year_opoy_players = set(opoy_df_aug[opoy_df_aug['Year'] == year]['Player_clean'].dropna())
        print(f"OPOY players in {year}: {sorted(year_opoy_players)}")

        # Fetch passing data
        passing_data = fetch_passing(year)
        if passing_data is not None:
            passing_data['Player_clean'] = passing_data['Player']

            if debug:
                pfr_players = set(passing_data['Player_clean'].dropna())
                overlap = year_opoy_players.intersection(pfr_players)
                print(f"  Passing overlap: {len(overlap)}/{len(year_opoy_players)} OPOY players found")
                if overlap:
                    print(f"    Matched: {sorted(overlap)}")

            pass_frames.append(passing_data)
            print(f"  Passing data: {passing_data.shape[0]} players, {passing_data.shape[1]} columns")

        time.sleep(sleep_time)

        # Fetch scrimmage data
        scrimmage_data = fetch_scrimmage(year)
        if scrimmage_data is not None:
            scrimmage_data['Player_clean'] = scrimmage_data['Player']

            if debug:
                pfr_players = set(scrimmage_data['Player_clean'].dropna())
                overlap = year_opoy_players.intersection(pfr_players)
                print(f"  Scrimmage overlap: {len(overlap)}/{len(year_opoy_players)} OPOY players found")
                if overlap:
                    print(f"    Matched: {sorted(overlap)}")

            scrim_frames.append(scrimmage_data)
            print(f"  Scrimmage data: {scrimmage_data.shape[0]} players, {scrimmage_data.shape[1]} columns")

        time.sleep(sleep_time)

    # Combine all data
    pass_all = pd.concat(pass_frames, ignore_index=True) if pass_frames else pd.DataFrame()
    scrim_all = pd.concat(scrim_frames, ignore_index=True) if scrim_frames else pd.DataFrame()

    print(f"\n{'='*50}")
    print(f"SUMMARY:")
    print(f"Combined passing data: {pass_all.shape}")
    print(f"Combined scrimmage data: {scrim_all.shape}")

    # Merge with OPOY data
    merged = opoy_df_aug.copy()

    if not pass_all.empty:
        print(f"\nMerging passing data...")
        merged = merged.merge(
            pass_all.drop(columns=['Player'], errors='ignore'),
            on=['Year','Player_clean'],
            how='left',
            suffixes=('', '_pass_dup')
        )

    if not scrim_all.empty:
        print(f"Merging scrimmage data...")
        merged = merged.merge(
            scrim_all.drop(columns=['Player'], errors='ignore'),
            on=['Year','Player_clean'],
            how='left',
            suffixes=('', '_scrim_dup')
        )

    merged = merged.drop(columns=['Player_clean'])

    # Convert numeric columns
    for col in merged.columns:
        if any(tag in col for tag in ['PFR_Pass_','PFR_Scrim_']):
            merged[col] = pd.to_numeric(merged[col], errors='ignore')

    print(f"\nFinal merged shape: {merged.shape}")

    return merged

In [ ]:
def clean_merged_opoy_data(df: pd.DataFrame, prefer_pfr: bool = True) -> pd.DataFrame:
    """
    Clean and consolidate the merged OPOY + PFR data.
    """
    print("Starting comprehensive data cleaning...")
    print(f"Original shape: {df.shape}")

    cleaned = df.copy()

    # Remove unnamed columns
    unwanted_cols = [col for col in cleaned.columns if col.startswith('Unnamed:') and ('level_0' in col or col == 'Unnamed: 0')]
    if unwanted_cols:
        cleaned = cleaned.drop(columns=unwanted_cols)

    # Remove per-game stats
    per_game_indicators = ['/G', 'per G', '_G_', 'Per_Game', 'Y_G', 'R_G', 'A_G']
    per_game_cols = [col for col in cleaned.columns if any(indicator in col for indicator in per_game_indicators)]
    if per_game_cols:
        cleaned = cleaned.drop(columns=per_game_cols)

    # Consolidate duplicate stat columns
    stat_mappings = {
        'Games': ['G', 'PFR_Pass_G', 'PFR_Scrim_G'],
        'Games_Started': ['GS', 'PFR_Pass_GS', 'PFR_Scrim_GS'],
        'Age': ['PFR_Pass_Age', 'PFR_Scrim_Age'],
        'Team': ['Team', 'PFR_Pass_Tm'],
        'Position': ['Position', 'PFR_Pass_Position', 'PFR_Scrim_Position'],
        'Pass_Completions': ['Pass_Cmp', 'PFR_Pass_Cmp'],
        'Pass_Attempts': ['Pass_Att', 'PFR_Pass_Att'],
        'Pass_Yards': ['Pass_Yds', 'PFR_Pass_Yds'],
        'Pass_Touchdowns': ['Pass_TD', 'PFR_Pass_TD'],
        'Pass_Interceptions': ['Pass_Int', 'PFR_Pass_Int'],
        'Rush_Attempts': ['Rush_Att', 'PFR_Scrim_Rushing_Att'],
        'Rush_Yards': ['Rush_Yds', 'PFR_Scrim_Rushing_Yds'],
        'Rush_Touchdowns': ['Rush_TD', 'PFR_Scrim_Rushing_TD'],
        'Rec_Receptions': ['Rec_Rec', 'PFR_Scrim_Receiving_Rec'],
        'Rec_Yards': ['Rec_Yds', 'PFR_Scrim_Receiving_Yds'],
        'Rec_Touchdowns': ['Rec_TD', 'PFR_Scrim_Receiving_TD'],
    }

    for final_name, source_cols in stat_mappings.items():
        available_cols = [col for col in source_cols if col in cleaned.columns]

        if len(available_cols) > 1:
            if prefer_pfr:
                pfr_cols = [col for col in available_cols if 'PFR_' in col]
                mvp_cols = [col for col in available_cols if 'PFR_' not in col]
                priority_order = pfr_cols + mvp_cols
            else:
                mvp_cols = [col for col in available_cols if 'PFR_' not in col]
                pfr_cols = [col for col in available_cols if 'PFR_' in col]
                priority_order = mvp_cols + pfr_cols

            cleaned[final_name] = pd.Series(dtype='object', index=cleaned.index)

            for col in reversed(priority_order):
                mask = cleaned[final_name].isna() & cleaned[col].notna()
                cleaned.loc[mask, final_name] = cleaned.loc[mask, col]

            cleaned = cleaned.drop(columns=available_cols)
        elif len(available_cols) == 1:
            old_name = available_cols[0]
            if old_name != final_name:
                cleaned = cleaned.rename(columns={old_name: final_name})

    # Keep useful PFR advanced stats
    keep_indicators = [
        'Rate', 'QBR', 'Y/A', 'AY/A', 'Y/C', 'NY/A', 'ANY/A',
        'Sk', '4QC', 'GWD', 'QBrec',
        'Tgt', 'Succ%', 'Lng', 'Ctch%', 'Y_R', 'Y_Tgt',
        'Touch', 'Y_Tch', '1D'
    ]

    removed_cols = []
    for col in list(cleaned.columns):
        if 'PFR_' in col and not any(indicator in col for indicator in keep_indicators):
            cleaned = cleaned.drop(columns=[col])
            removed_cols.append(col)

    # Clean up PFR column names
    name_cleanup = {}
    for col in cleaned.columns:
        if 'PFR_' in col:
            new_name = col.replace('PFR_Pass_', '').replace('PFR_Scrim_', '')
            new_name = new_name.replace('_', ' ').replace('/', ' per ').replace('%', ' Pct')
            new_name = re.sub(r'\s+', '_', new_name).strip('_')

            if any(pass_stat in col for pass_stat in ['Pass', 'Rate', 'QBR', 'Sk', 'QBrec', '4QC', 'GWD']):
                new_name = f"Pass_{new_name}"
            elif 'Receiving' in col or any(rec_stat in col for rec_stat in ['Tgt', 'Ctch', 'Rec']):
                new_name = f"Rec_{new_name}"
            elif 'Rushing' in col:
                new_name = f"Rush_{new_name}"
            elif 'Scrimmage' in col:
                new_name = f"Scrim_{new_name}"

            name_cleanup[col] = new_name

    if name_cleanup:
        cleaned = cleaned.rename(columns=name_cleanup)

    # Convert data types
    text_cols = ['Player', 'Team', 'Position']

    for col in cleaned.columns:
        if col not in text_cols and col != 'Year':
            cleaned[col] = pd.to_numeric(cleaned[col], errors='coerce')

    for col in text_cols:
        if col in cleaned.columns:
            cleaned[col] = cleaned[col].astype(str).str.strip()

    # Organize columns
    base_info = ['Year', 'Rank', 'Player', 'Team', 'Position', 'Votes']
    game_info = ['Games', 'Games_Started', 'Age']

    passing_stats = [col for col in cleaned.columns if col.startswith('Pass_')]
    rushing_stats = [col for col in cleaned.columns if col.startswith('Rush_')]
    receiving_stats = [col for col in cleaned.columns if col.startswith('Rec_')]
    scrimmage_stats = [col for col in cleaned.columns if col.startswith('Scrim_')]
    defense_stats = [col for col in cleaned.columns if col.startswith('Def_')]

    accounted_for = set(base_info + game_info + passing_stats + rushing_stats +
                       receiving_stats + scrimmage_stats + defense_stats)
    other_cols = [col for col in cleaned.columns if col not in accounted_for]

    final_order = []
    for group in [base_info, game_info, passing_stats, rushing_stats,
                  receiving_stats, scrimmage_stats, defense_stats, other_cols]:
        final_order.extend([col for col in group if col in cleaned.columns])

    cleaned = cleaned[final_order]

    print(f"\nFinal shape: {cleaned.shape}")
    print(f"Columns removed: {df.shape[1] - cleaned.shape[1]}")
    print(f"Final columns: {len(cleaned.columns)}")

    return cleaned

In [ ]:
# Step 1: Clean the scraped OPOY data
opoy_df_clean = clean_opoy_data(opoy_df)

# Step 2: Merge with PFR stats (this will take time due to web scraping)
opoy_advanced = merge_opoy_with_pfr(opoy_df_clean, sleep_time = 3, debug = False)

# Step 3: Final cleanup and consolidation
opoy_final = clean_merged_opoy_data(opoy_advanced, prefer_pfr=True)

# Step 4: Save the results
opoy_final.to_csv('opoy_complete_dataset.csv', index=False)

print("\n" + "="*50)
print("OPOY DATA PROCESSING COMPLETE!")
print(f"Final dataset shape: {opoy_final.shape}")
print(f"Years covered: {opoy_final['Year'].min()} - {opoy_final['Year'].max()}")
print(f"Total players: {opoy_final['Player'].nunique()}")
print("\nFirst few rows:")
print(opoy_final.head())

In [ ]:
# advanced OPOY
opoy_advanced.to_csv('opoy_advanced.csv')
files.download('opoy_advanced.csv')

In [ ]:
# Skip Cleaning
opoy_adv_clean = '/content/drive/MyDrive/CS3540/Project/opoy_adv_clean.csv'
opoy_adv_clean = pd.read_csv(opoy_adv_clean, index_col = 0)
opoy_adv_clean.head(50)

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/CS3540/Project/opoy_adv_clean.csv'

In [ ]:
def clean_opoy_advanced_improved(df: pd.DataFrame, prefer_pfr: bool = True) -> pd.DataFrame:
    """
    Improved cleaning for OPOY merged data with better handling of messy columns.
    """
    print("Starting improved OPOY data cleaning...")
    print(f"Original shape: {df.shape}")

    cleaned = df.copy()

    # Step 1: Fix messy PFR_Scrim column names FIRST
    print("\nStep 1: Fixing messy PFR_Scrim columns")
    name_fixes = {}
    for col in cleaned.columns:
        if 'PFR_Scrim_Unnamed' in col:
            if 'Age' in col:
                name_fixes[col] = 'PFR_Scrim_Age'
            elif 'Pos' in col:
                name_fixes[col] = 'PFR_Scrim_Position'
            elif col.endswith('_G') and 'GS' not in col:
                name_fixes[col] = 'PFR_Scrim_G'
            elif 'GS' in col:
                name_fixes[col] = 'PFR_Scrim_GS'

    if name_fixes:
        cleaned = cleaned.rename(columns=name_fixes)
        print(f"Fixed {len(name_fixes)} messy column names")

    # Step 2: Remove unnamed index columns
    print("\nStep 2: Removing unnamed columns")
    unwanted = [col for col in cleaned.columns if col.startswith('Unnamed:')]
    if unwanted:
        cleaned = cleaned.drop(columns=unwanted)
        print(f"Removed {len(unwanted)} unnamed columns")

    # Step 3: DROP ALL DEFENSIVE STATS (this is OPOY - offense only!)
    print("\nStep 3: Removing defensive stats (OPOY is offense-only)")
    def_cols = [col for col in cleaned.columns if col.startswith('Def_')]
    if def_cols:
        cleaned = cleaned.drop(columns=def_cols)
        print(f"Removed {len(def_cols)} defensive columns: {def_cols}")

    # Step 4: Remove per-game stats
    print("\nStep 4: Removing per-game statistics")
    per_game_cols = [col for col in cleaned.columns if any(x in col for x in ['/G', '_G_', 'Per_Game', 'Y_G', 'R_G', 'A_G'])]
    if per_game_cols:
        cleaned = cleaned.drop(columns=per_game_cols)
        print(f"Removed {len(per_game_cols)} per-game columns")

    # Step 5: Consolidate duplicates
    print("\nStep 5: Consolidating duplicate columns")

    stat_mappings = {
        'Games': ['G', 'PFR_Pass_G', 'PFR_Scrim_G'],
        'Games_Started': ['GS', 'PFR_Pass_GS', 'PFR_Scrim_GS'],
        'Age': ['PFR_Pass_Age', 'PFR_Scrim_Age'],
        'Pass_Completions': ['Pass_Cmp', 'PFR_Pass_Cmp'],
        'Pass_Attempts': ['Pass_Att', 'PFR_Pass_Att'],
        'Pass_Yards': ['Pass_Yds', 'PFR_Pass_Yds'],
        'Pass_Touchdowns': ['Pass_TD', 'PFR_Pass_TD'],
        'Pass_Interceptions': ['Pass_Int', 'PFR_Pass_Int'],
        'Rush_Attempts': ['Rush_Att', 'PFR_Scrim_Rushing_Att'],
        'Rush_Yards': ['Rush_Yds', 'PFR_Scrim_Rushing_Yds'],
        'Rush_Touchdowns': ['Rush_TD', 'PFR_Scrim_Rushing_TD'],
        'Rec_Receptions': ['Rec_Rec', 'PFR_Scrim_Receiving_Rec'],
        'Rec_Yards': ['Rec_Yds', 'PFR_Scrim_Receiving_Yds'],
        'Rec_Touchdowns': ['Rec_TD', 'PFR_Scrim_Receiving_TD'],
    }

    consolidated = 0
    for final_name, source_cols in stat_mappings.items():
        available = [col for col in source_cols if col in cleaned.columns]

        if len(available) > 1:
            pfr_cols = [col for col in available if 'PFR_' in col]
            other_cols = [col for col in available if 'PFR_' not in col]
            priority = pfr_cols + other_cols if prefer_pfr else other_cols + pfr_cols

            cleaned[final_name] = pd.Series(dtype='object', index=cleaned.index)
            for col in reversed(priority):
                mask = cleaned[final_name].isna() & cleaned[col].notna()
                cleaned.loc[mask, final_name] = cleaned.loc[mask, col]

            cleaned = cleaned.drop(columns=available)
            consolidated += 1

        elif len(available) == 1 and available[0] != final_name:
            cleaned = cleaned.rename(columns={available[0]: final_name})

    # Drop redundant PFR Position columns but keep the original Position
    pfr_pos_cols = ['PFR_Pass_Position', 'PFR_Scrim_Position']
    pfr_pos_to_drop = [col for col in pfr_pos_cols if col in cleaned.columns]
    if pfr_pos_to_drop:
        cleaned = cleaned.drop(columns=pfr_pos_to_drop)

    print(f"Consolidated {consolidated} stat types")

    # Step 6: Keep only useful advanced PFR stats
    print("\nStep 6: Filtering advanced stats")
    keep_patterns = ['Rate', 'QBR', 'Y/A', 'AY/A', 'Y/C', 'NY/A', 'ANY/A',
                     'Sk', '4QC', 'GWD', 'QBrec', 'Tgt', 'Succ%', 'Lng',
                     'Ctch%', 'Y_R', 'Y_Tgt', 'Touch', 'Y_Tch', '1D', 'YScm', 'RRTD']

    pfr_cols_to_keep = []
    pfr_cols_to_remove = []

    for col in cleaned.columns:
        if 'PFR_' in col:
            if any(pattern in col for pattern in keep_patterns):
                pfr_cols_to_keep.append(col)
            else:
                pfr_cols_to_remove.append(col)

    if pfr_cols_to_remove:
        cleaned = cleaned.drop(columns=pfr_cols_to_remove)
        print(f"Removed {len(pfr_cols_to_remove)} redundant PFR columns")
        print(f"Kept {len(pfr_cols_to_keep)} advanced stat columns")

    # Step 7: Clean up remaining PFR column names
    print("\nStep 7: Cleaning PFR column names")
    rename_map = {}
    for col in cleaned.columns:
        if 'PFR_' in col:
            new_name = col.replace('PFR_Pass_', 'Pass_').replace('PFR_Scrim_', '')
            new_name = new_name.replace('Receiving_', 'Rec_').replace('Rushing_', 'Rush_')
            new_name = new_name.replace('Scrimmage_', 'Scrim_')
            new_name = new_name.replace('/', '_per_').replace('%', '_Pct')
            rename_map[col] = new_name

    if rename_map:
        cleaned = cleaned.rename(columns=rename_map)
        print(f"Cleaned {len(rename_map)} PFR column names")

    # Step 8: Convert data types
    print("\nStep 8: Converting data types")
    text_cols = ['Player', 'Team', 'Position']
    for col in cleaned.columns:
        if col not in text_cols and col != 'Year':
            cleaned[col] = pd.to_numeric(cleaned[col], errors='coerce')

    for col in text_cols:
        if col in cleaned.columns:
            cleaned[col] = cleaned[col].astype(str).str.strip()

    # Step 9: Organize columns (NO DEFENSIVE STATS)
    print("\nStep 9: Organizing columns")
    base = ['Year', 'Rank', 'Player', 'Team', 'Position', 'Votes']
    game = ['Games', 'Games_Started', 'Age']
    passing = sorted([c for c in cleaned.columns if c.startswith('Pass_')])
    rushing = sorted([c for c in cleaned.columns if c.startswith('Rush_')])
    receiving = sorted([c for c in cleaned.columns if c.startswith('Rec_')])
    scrimmage = sorted([c for c in cleaned.columns if c.startswith('Scrim_')])

    all_organized = set(base + game + passing + rushing + receiving + scrimmage)
    other = [c for c in cleaned.columns if c not in all_organized]

    final_order = []
    for group in [base, game, passing, rushing, receiving, scrimmage, other]:
        final_order.extend([c for c in group if c in cleaned.columns])

    cleaned = cleaned[final_order]

    print(f"\n{'='*50}")
    print("CLEANING COMPLETE!")
    print(f"Final shape: {cleaned.shape}")
    print(f"Removed {df.shape[1] - cleaned.shape[1]} columns")

    return cleaned

# Re-clean with defensive stats removed
opoy_df_adv = pd.read_csv('/content/drive/MyDrive/CS3540/Project/opoy_df_adv.csv', index_col=0)
opoy_adv_clean = clean_opoy_advanced_improved(opoy_df_adv, prefer_pfr=True)

# Save
opoy_adv_clean.to_csv('opoy_adv_clean.csv')
files.download('opoy_adv_clean.csv')

# Verify no defensive columns remain
print("\nFinal columns:")
print(opoy_adv_clean.columns.tolist())
print(f"\nAny 'Def_' columns remaining? {any('Def_' in col for col in opoy_adv_clean.columns)}")

print("\nSample data:")
print(opoy_adv_clean[['Year', 'Player', 'Position', 'Votes', 'Pass_Yards', 'Rush_Yards', 'Rec_Yards']].head(5))

Starting improved OPOY data cleaning...
Original shape: (277, 71)

Step 1: Fixing messy PFR_Scrim columns
Fixed 4 messy column names

Step 2: Removing unnamed columns

Step 3: Removing defensive stats (OPOY is offense-only)
Removed 1 defensive columns: ['Def_Int']

Step 4: Removing per-game statistics
Removed 5 per-game columns

Step 5: Consolidating duplicate columns
Consolidated 14 stat types

Step 6: Filtering advanced stats
Removed 2 redundant PFR columns
Kept 25 advanced stat columns

Step 7: Cleaning PFR column names
Cleaned 25 PFR column names

Step 8: Converting data types

Step 9: Organizing columns

CLEANING COMPLETE!
Final shape: (277, 45)
Removed 26 columns


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


Final columns:
['Year', 'Rank', 'Player', 'Team', 'Position', 'Votes', 'Games', 'Games_Started', 'Age', 'Pass_4QC', 'Pass_ANY_per_A', 'Pass_AY_per_A', 'Pass_Attempts', 'Pass_Completions', 'Pass_GWD', 'Pass_Interceptions', 'Pass_NY_per_A', 'Pass_QBR', 'Pass_QBrec', 'Pass_Rate', 'Pass_Sk', 'Pass_Touchdowns', 'Pass_Y_per_A', 'Pass_Y_per_C', 'Pass_Yards', 'Rush_1D', 'Rush_Attempts', 'Rush_Lng', 'Rush_Succ_Pct', 'Rush_Touchdowns', 'Rush_Yards', 'Rec_1D', 'Rec_Ctch_Pct', 'Rec_Lng', 'Rec_Receptions', 'Rec_Succ_Pct', 'Rec_Tgt', 'Rec_Touchdowns', 'Rec_Y_R', 'Rec_Y_Tgt', 'Rec_Yards', 'Scrim_RRTD', 'Scrim_Touch', 'Scrim_YScm', 'Scrim_Y_Tch']

Any 'Def_' columns remaining? False

Sample data:
   Year          Player Position  Votes  Pass_Yards  Rush_Yards  Rec_Yards
0  2024  Saquon Barkley       RB   94.0           0        2005        278
1  2024   Lamar Jackson       QB   43.0        4172         915          0
2  2024   Ja'Marr Chase       WR   40.0           0          32       1708
3  2024  

# DPOY Data

In [ ]:
# Step 1: Scrape DPOY data
print("="*60)
print("SCRAPING DPOY DATA")
print("="*60)

year = 2024
dpoy_dfs = []

while year >= 1980:
    url = f'https://www.pro-football-reference.com/awards/awards_{year}.htm'
    try:
        # Get all tables from the page
        tables = pd.read_html(url)

        # tables[2] is DPOY voting
        dpoy_table = tables[2]

        # Check if columns are multi-level and handle accordingly
        if isinstance(dpoy_table.columns, pd.MultiIndex):
            dpoy_table.columns = [' '.join(str(col).strip() for col in multi_col if str(col) != 'nan')
                                   for multi_col in dpoy_table.columns]
            dpoy_table['Year'] = year
        else:
            dpoy_table['Year'] = year

        dpoy_dfs.append(dpoy_table)
        print(f"Scraped DPOY {year} - Shape: {dpoy_table.shape}")

    except Exception as e:
        print(f"Failed to scrape DPOY {year}: {e}")

    time.sleep(3)
    year -= 1

# Combine all dataframes
dpoy_df = pd.concat(dpoy_dfs, ignore_index=True)
print(f"\nDPOY Total rows: {len(dpoy_df)}")
print(f"Years covered: {dpoy_df['Year'].min()} - {dpoy_df['Year'].max()}")

# Save raw scraped data
dpoy_df.to_csv('dpoy_df.csv')
files.download('dpoy_df.csv')

print("\nSample DPOY data:")
print(dpoy_df.head(10))

SCRAPING DPOY DATA
Scraped DPOY 2024 - Shape: (24, 16)
Scraped DPOY 2023 - Shape: (11, 16)
Scraped DPOY 2022 - Shape: (10, 16)
Scraped DPOY 2021 - Shape: (3, 15)
Scraped DPOY 2020 - Shape: (3, 15)
Scraped DPOY 2019 - Shape: (7, 15)
Scraped DPOY 2018 - Shape: (2, 15)
Scraped DPOY 2017 - Shape: (8, 15)
Scraped DPOY 2016 - Shape: (6, 15)
Scraped DPOY 2015 - Shape: (4, 15)
Scraped DPOY 2014 - Shape: (1, 15)
Scraped DPOY 2013 - Shape: (6, 15)
Scraped DPOY 2012 - Shape: (2, 15)
Scraped DPOY 2011 - Shape: (7, 15)
Scraped DPOY 2010 - Shape: (7, 15)
Scraped DPOY 2009 - Shape: (5, 15)
Scraped DPOY 2008 - Shape: (5, 15)
Scraped DPOY 2007 - Shape: (10, 15)
Scraped DPOY 2006 - Shape: (6, 15)
Scraped DPOY 2005 - Shape: (10, 15)
Scraped DPOY 2004 - Shape: (6, 15)
Scraped DPOY 2003 - Shape: (6, 15)
Scraped DPOY 2002 - Shape: (3, 15)
Scraped DPOY 2001 - Shape: (4, 15)
Scraped DPOY 2000 - Shape: (6, 15)
Scraped DPOY 1999 - Shape: (7, 15)
Scraped DPOY 1998 - Shape: (15, 15)
Scraped DPOY 1997 - Shape: (5,

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


Sample DPOY data:
   Unnamed: 0_level_0 Rk Unnamed: 1_level_0 Pos Unnamed: 2_level_0 Player  \
0                      1                     CB        Patrick Surtain II   
1                      2                     DE          Trey Hendrickson   
2                      3                     DE             Myles Garrett   
3                      4                     LB                 T.J. Watt   
4                      5                     LB                 Zack Baun   
5                      6                      S              Kerby Joseph   
6                      7                    OLB         Andrew Van Ginkel   
7                      8                      S           Xavier McKinney   
8                      9                    OLB               Nik Bonitto   
9                     10                      S             Kyle Hamilton   

  Unnamed: 3_level_0 Tm  Unnamed: 4_level_0 Vote Pts  \
0        Denver Broncos                        320.0   
1    Cincinnati Benga

In [ ]:
# Step 2: Clean DPOY data
def clean_dpoy_data(dpoy_df):
    """
    Clean the scraped DPOY DataFrame.
    Similar to MVP/OPOY cleaning but focused on defensive stats.
    """
    print("Starting DPOY data cleaning...")
    print(f"Original shape: {dpoy_df.shape}")

    dpoy_df_clean = dpoy_df.copy()

    # Drop unnecessary columns
    dpoy_df_clean = dpoy_df_clean.drop(columns=["Unnamed: 0"], errors="ignore")

    # Rename columns
    column_mapping = {
        "Unnamed: 0_level_0 Rk": "Rank",
        "Unnamed: 1_level_0 Pos": "Position",
        "Unnamed: 2_level_0 Player": "Player",
        "Unnamed: 3_level_0 Tm": "Team",
        "Unnamed: 4_level_0 Vote Pts": "VotePoints_1",
        "Unnamed: 4_level_0 Votes": "VotePoints_2",
        "Unnamed: 5_level_0 1st Place": "FirstPlaceVotes",
        "Unnamed: 6_level_0 Share": "VoteShare_1",
        "Unnamed: 5_level_0 Share": "VoteShare_2",
        "Games G": "G",
        "Games GS": "GS",
        # Defensive stats
        "Defense Int": "Def_Int",
        "Defense Yds": "Def_Int_Yds",
        "Defense TD": "Def_Int_TD",
        "Defense PD": "Def_PD",
        "Defense Sk": "Def_Sk",
        "Defense Comb": "Def_Comb",
        "Defense Solo": "Def_Solo",
        "Defense Ast": "Def_Ast",
        "Defense TFL": "Def_TFL",
        "Defense QBHits": "Def_QBHits",
        "Defense FR": "Def_FR",
        "Defense FF": "Def_FF",
    }

    dpoy_df_clean = dpoy_df_clean.rename(columns=column_mapping)

    # Consolidate voting columns
    vote_points_cols = [c for c in ["VotePoints_1", "VotePoints_2"] if c in dpoy_df_clean.columns]
    if vote_points_cols:
        dpoy_df_clean["VotePoints"] = pd.to_numeric(dpoy_df_clean[vote_points_cols[0]], errors="coerce")
        for col in vote_points_cols[1:]:
            dpoy_df_clean["VotePoints"] = dpoy_df_clean["VotePoints"].fillna(
                pd.to_numeric(dpoy_df_clean[col], errors="coerce")
            )
        dpoy_df_clean = dpoy_df_clean.drop(columns=vote_points_cols)

    # Convert FirstPlaceVotes to numeric
    if "FirstPlaceVotes" in dpoy_df_clean.columns:
        dpoy_df_clean["FirstPlaceVotes"] = pd.to_numeric(dpoy_df_clean["FirstPlaceVotes"], errors="coerce")

    # Create unified Votes column (same logic as MVP/OPOY)
    if "VotePoints" in dpoy_df_clean.columns and "Year" in dpoy_df_clean.columns:
        votes_2022_plus = dpoy_df_clean["VotePoints"] / 4.2
        votes_2022_plus = np.where(pd.isna(votes_2022_plus), np.nan, np.round(votes_2022_plus))
        votes_2022_plus = np.where(
            (votes_2022_plus == 0) & (dpoy_df_clean["VotePoints"] > 0),
            1,
            votes_2022_plus
        )

        dpoy_df_clean["Votes"] = np.where(
            dpoy_df_clean["Year"] >= 2022,
            votes_2022_plus,
            dpoy_df_clean["VotePoints"]
        )

    # Convert stat columns to numeric
    numeric_columns = [
        "Rank", "VotePoints", "FirstPlaceVotes", "Votes",
        "G", "GS", "Year",
        "Def_Int", "Def_Int_Yds", "Def_Int_TD", "Def_PD", "Def_Sk",
        "Def_Comb", "Def_Solo", "Def_Ast", "Def_TFL", "Def_QBHits",
        "Def_FR", "Def_FF"
    ]

    for col in numeric_columns:
        if col in dpoy_df_clean.columns:
            dpoy_df_clean[col] = pd.to_numeric(dpoy_df_clean[col], errors="coerce")

    # Clean text columns
    for col in ["Position", "Player", "Team"]:
        if col in dpoy_df_clean.columns:
            dpoy_df_clean[col] = dpoy_df_clean[col].astype(str).str.strip()

    # Drop unwanted columns
    cols_to_drop = ["VoteShare_1", "VoteShare_2", "VotePoints", "FirstPlaceVotes"]
    dpoy_df_clean = dpoy_df_clean.drop(
        columns=[c for c in cols_to_drop if c in dpoy_df_clean.columns],
        errors="ignore"
    )

    # Reorder columns
    preferred_order = [
        "Year", "Rank", "Player", "Team", "Position", "Votes",
        "G", "GS",
        "Def_Int", "Def_Int_Yds", "Def_Int_TD", "Def_PD", "Def_Sk",
        "Def_Comb", "Def_Solo", "Def_Ast", "Def_TFL", "Def_QBHits",
        "Def_FR", "Def_FF"
    ]

    final_columns = [c for c in preferred_order if c in dpoy_df_clean.columns]
    final_columns.extend([c for c in dpoy_df_clean.columns if c not in final_columns])
    dpoy_df_clean = dpoy_df_clean[final_columns]

    print(f"Cleaned shape: {dpoy_df_clean.shape}")
    print(f"Columns: {dpoy_df_clean.columns.tolist()}")

    return dpoy_df_clean

# Apply cleaning
dpoy_df_clean = clean_dpoy_data(dpoy_df)

# Save cleaned data
dpoy_df_clean.to_csv('dpoy_df_clean.csv')
files.download('dpoy_df_clean.csv')

print("\nSample cleaned DPOY data:")
print(dpoy_df_clean.head())

Starting DPOY data cleaning...
Original shape: (331, 20)
Cleaned shape: (331, 16)
Columns: ['Year', 'Rank', 'Player', 'Team', 'Position', 'Votes', 'G', 'GS', 'Tackles Solo', 'Tackles Ast', 'Unnamed: 11_level_0 Sk', 'Def Interceptions Int', 'Def Interceptions Yds', 'Def Interceptions TD', 'Unnamed: 10_level_0 Sk', 'Unnamed: 8_level_0 Sk']


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


Sample cleaned DPOY data:
   Year  Rank              Player                 Team Position  Votes   G  \
0  2024     1  Patrick Surtain II       Denver Broncos       CB   76.0  16   
1  2024     2    Trey Hendrickson   Cincinnati Bengals       DE   48.0  17   
2  2024     3       Myles Garrett     Cleveland Browns       DE   37.0  17   
3  2024     4           T.J. Watt  Pittsburgh Steelers       LB   26.0  17   
4  2024     5           Zack Baun  Philadelphia Eagles       LB   15.0  16   

   GS  Tackles Solo  Tackles Ast  Unnamed: 11_level_0 Sk  \
0  16          34.0         11.0                     0.0   
1  17          33.0         13.0                    17.5   
2  17          40.0          7.0                    14.0   
3  17          40.0         21.0                    11.5   
4  16          93.0         58.0                     3.5   

   Def Interceptions Int  Def Interceptions Yds  Def Interceptions TD  \
0                      4                    132                     1 

In [ ]:
# Updated helper functions
def _flatten_cols(df: pd.DataFrame) -> pd.DataFrame:
    """Flatten multi-level column headers."""
    if isinstance(df.columns, pd.MultiIndex):
        new_cols = []
        for col in df.columns:
            # Join non-empty parts of multi-level column names
            parts = [str(c).strip() for c in col if str(c) != 'nan' and str(c).strip() and 'Unnamed:' not in str(c)]
            if not parts:
                # If all parts were unnamed, check if we can use the last level
                last_part = str(col[-1]).strip()
                if last_part and last_part != 'nan':
                    new_cols.append(last_part)
                else:
                    new_cols.append('_'.join(str(c) for c in col))
            else:
                new_cols.append(' '.join(parts))
        df.columns = new_cols
    return df

def _clean_player_name(name: str) -> str:
    """Clean player name by removing special characters and standardizing format."""
    if pd.isna(name):
        return ""
    name = str(name).strip()
    # Remove asterisks, plus signs, and other special characters
    name = name.replace('*', '').replace('+', '').replace('HOF', '').strip()
    return name

def _pick_main_table(tables: list) -> Optional[pd.DataFrame]:
    """Pick the main stats table from a list of tables."""
    if not tables:
        return None

    # Usually the main table is the first or second one
    for table in tables[:3]:
        if table.shape[0] > 10:  # Has reasonable number of rows
            return table

    # Fallback to first table
    return tables[0] if tables else None

def _prefer_totals_rows(df: pd.DataFrame) -> pd.DataFrame:
    """
    When a player appears multiple times (played for multiple teams),
    prefer the row with 'Tm' == '2TM', '3TM', etc. or the TOT row.
    """
    if 'Player' not in df.columns:
        return df

    team_col = None
    for possible_team_col in ['Tm', 'Team']:
        if possible_team_col in df.columns:
            team_col = possible_team_col
            break

    if team_col is None:
        return df

    # Group by Player and Year
    year_col = 'Year' if 'Year' in df.columns else None

    if year_col:
        group_cols = ['Player', 'Year']
    else:
        group_cols = ['Player']

    result_rows = []

    for name, group in df.groupby(group_cols, dropna=False):
        if len(group) == 1:
            result_rows.append(group.iloc[0].to_dict())
        else:
            # Prefer TOT, 2TM, 3TM, etc.
            total_rows = group[group[team_col].astype(str).str.contains('TM', na=False)]
            if not total_rows.empty:
                result_rows.append(total_rows.iloc[0].to_dict())
            else:
                # Just take the first one
                result_rows.append(group.iloc[0].to_dict())

    return pd.DataFrame(result_rows)

# UPDATED fetch_defense function with better column handling
def fetch_defense(year: int, max_retries: int = 3) -> Optional[pd.DataFrame]:
    """Fetch defensive stats from Pro Football Reference."""
    url = f'https://www.pro-football-reference.com/years/{year}/defense.htm'

    for attempt in range(max_retries):
        try:
            tables = pd.read_html(url)
            t = _pick_main_table(tables)
            if t is None:
                print(f"No suitable defense table found for {year}")
                return None

            # Flatten columns before processing
            t = _flatten_cols(t)

            # Print original columns for debugging
            print(f" Defense columns for {year}: {t.columns.tolist()}")

            # Select and rename specific columns to avoid duplicates
            column_map = {}

            for col in t.columns:
                col_str = str(col)

                # Map to standardized names
                if col_str == 'Player':
                    column_map[col] = 'Player'
                elif col_str in ['Team', 'Tm']:
                    column_map[col] = 'Tm'
                elif col_str == 'Age':
                    column_map[col] = 'Age'
                elif col_str in ['Pos', 'Position']:
                    column_map[col] = 'Pos'
                elif col_str == 'G':
                    column_map[col] = 'G'
                elif col_str == 'GS':
                    column_map[col] = 'GS'
                elif col_str == 'Def Interceptions Int':
                    column_map[col] = 'Int'
                elif col_str == 'Def Interceptions Yds':
                    column_map[col] = 'Int_Yds'
                elif col_str == 'Def Interceptions IntTD':
                    column_map[col] = 'Int_TD'
                elif col_str == 'Def Interceptions PD':
                    column_map[col] = 'PD'
                elif col_str == 'Sk':
                    column_map[col] = 'Sk'
                elif col_str == 'Tackles Comb':
                    column_map[col] = 'Comb'
                elif col_str == 'Tackles Solo':
                    column_map[col] = 'Solo'
                elif col_str == 'Tackles Ast':
                    column_map[col] = 'Ast'
                elif col_str == 'Tackles TFL':
                    column_map[col] = 'TFL'
                elif col_str == 'Tackles QBHits':
                    column_map[col] = 'QBHits'
                elif col_str == 'Fumbles FR':
                    column_map[col] = 'FR'
                elif col_str == 'Fumbles FF':
                    column_map[col] = 'FF'
                elif col_str == 'Sfty':
                    column_map[col] = 'Sfty'

            # Only keep columns we've mapped
            if 'Player' not in column_map.values():
                print(f"WARNING: No Player column found for {year}")
                return None

            # Select only the columns we want and rename them
            cols_to_keep = list(column_map.keys())
            t = t[cols_to_keep].copy()
            t = t.rename(columns=column_map)

            # Check for duplicate columns after renaming
            if t.columns.duplicated().any():
                print(f"WARNING: Duplicate columns detected for {year}: {t.columns[t.columns.duplicated()].tolist()}")
                # Keep only first occurrence of each column
                t = t.loc[:, ~t.columns.duplicated()]

            t['Year'] = year

            # Clean player names
            t['Player'] = t['Player'].apply(_clean_player_name)

            t = _prefer_totals_rows(t)

            # Add prefix to avoid conflicts with DPOY data
            rename_map = {}
            for col in t.columns:
                if col not in ['Player', 'Year']:
                    rename_map[col] = f'PFR_Def_{col}'
            t = t.rename(columns=rename_map)

            print(f" Final defense columns: {t.columns.tolist()}")
            print(f" Shape: {t.shape}")
            return t

        except Exception as e:
            print(f"Defense fetch attempt {attempt + 1}/{max_retries} failed for {year}: {e}")
            import traceback
            traceback.print_exc()
            if attempt < max_retries - 1:
                time.sleep(2)
            else:
                return None

# Now run the merge
dpoy_df_adv = merge_dpoy_with_pfr(dpoy_df_clean, sleep_time=3, debug=False)

# Save
dpoy_df_adv.to_csv('dpoy_df_adv.csv')
files.download('dpoy_df_adv.csv')

print("\nFirst few rows of merged DPOY data:")
print(dpoy_df_adv.head())

Processing years: [2024, 2023, 2022, 2021, 2020, 2019, 2018, 2017, 2016, 2015, 2014, 2013, 2012, 2011, 2010, 2009, 2008, 2007, 2006, 2005, 2004, 2003, 2002, 2001, 2000, 1999, 1998, 1997, 1996, 1995, 1994, 1993, 1992, 1991, 1990, 1989, 1988, 1987, 1986, 1985, 1984, 1983, 1982, 1981, 1980]

Processing year 2024 (1/45)...
DPOY players in 2024: ['Andrew Van Ginkel', 'Bobby Wagner', 'Cameron Heyward', 'Chris Jones', 'Christian Benford', 'Danielle Hunter', 'Derek Stingley Jr.', 'Jalen Carter', 'Jonathan Greenard', 'Kerby Joseph', 'Kyle Hamilton', 'Leonard Williams', 'Micah Parsons', 'Myles Garrett', 'Nik Bonitto', 'Patrick Surtain II', 'T.J. Watt', 'Trent McDuffie', 'Trey Hendrickson', 'Will Anderson', 'Xavier McKinney', 'Zach Allen', 'Zack Baun', 'Zaire Franklin']
 Defense columns for 2024: ['Rk', 'Player', 'Age', 'Team', 'Pos', 'G', 'GS', 'Def Interceptions Int', 'Def Interceptions Yds', 'Def Interceptions IntTD', 'Def Interceptions Lng', 'Def Interceptions PD', 'Fumbles FF', 'Fumbles Fmb'

/tmp/ipython-input-999914109.py:115: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  merged[col] = pd.to_numeric(merged[col], errors='ignore')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


First few rows of merged DPOY data:
   Year  Rank              Player                 Team Position  Votes   G  \
0  2024     1  Patrick Surtain II       Denver Broncos       CB   76.0  16   
1  2024     2    Trey Hendrickson   Cincinnati Bengals       DE   48.0  17   
2  2024     3       Myles Garrett     Cleveland Browns       DE   37.0  17   
3  2024     4           T.J. Watt  Pittsburgh Steelers       LB   26.0  17   
4  2024     5           Zack Baun  Philadelphia Eagles       LB   15.0  16   

   GS  Tackles Solo  Tackles Ast  ...  PFR_Def_PD  PFR_Def_FF  PFR_Def_FR  \
0  16          34.0         11.0  ...        11.0         1.0         1.0   
1  17          33.0         13.0  ...         6.0         2.0         0.0   
2  17          40.0          7.0  ...         1.0         3.0         1.0   
3  17          40.0         21.0  ...         4.0         6.0         2.0   
4  16          93.0         58.0  ...         4.0         5.0         1.0   

   PFR_Def_Sk  PFR_Def_Comb  PF

In [ ]:
# Step 4: Final cleanup of merged DPOY data
def final_cleanup_dpoy(dpoy_df_adv):
    """
    Final cleanup of merged DPOY dataframe to remove redundant columns
    and consolidate overlapping data.
    """
    print("Starting final DPOY cleanup...")
    print(f"Original shape: {dpoy_df_adv.shape}")

    df = dpoy_df_adv.copy()

    # Drop redundant columns from original DPOY scrape (now have PFR versions)
    columns_to_drop = [
        'Tackles Solo', 'Tackles Ast',
        'Unnamed: 11_level_0 Sk', 'Unnamed: 10_level_0 Sk', 'Unnamed: 8_level_0 Sk',
        'Def Interceptions Int', 'Def Interceptions Yds', 'Def Interceptions TD'
    ]

    df = df.drop(columns=[c for c in columns_to_drop if c in df.columns], errors='ignore')

    # Consolidate G and GS (prefer PFR version if available)
    if 'PFR_Def_G' in df.columns and 'G' in df.columns:
        df['G'] = df['PFR_Def_G'].fillna(df['G'])
        df = df.drop(columns=['PFR_Def_G'], errors='ignore')
    elif 'PFR_Def_G' in df.columns:
        df = df.rename(columns={'PFR_Def_G': 'G'})

    if 'PFR_Def_GS' in df.columns and 'GS' in df.columns:
        df['GS'] = df['PFR_Def_GS'].fillna(df['GS'])
        df = df.drop(columns=['PFR_Def_GS'], errors='ignore')
    elif 'PFR_Def_GS' in df.columns:
        df = df.rename(columns={'PFR_Def_GS': 'GS'})

    # Consolidate Team and Position (prefer original DPOY version)
    if 'PFR_Def_Tm' in df.columns:
        if 'Team' in df.columns:
            df['Team'] = df['Team'].fillna(df['PFR_Def_Tm'])
        df = df.drop(columns=['PFR_Def_Tm'], errors='ignore')

    if 'PFR_Def_Pos' in df.columns:
        if 'Position' in df.columns:
            df['Position'] = df['Position'].fillna(df['PFR_Def_Pos'])
        df = df.drop(columns=['PFR_Def_Pos'], errors='ignore')

    # Rename PFR columns to cleaner names (remove PFR_Def_ prefix for main stats)
    rename_map = {
        'PFR_Def_Age': 'Age',
        'PFR_Def_Int': 'Int',
        'PFR_Def_Int_Yds': 'Int_Yds',
        'PFR_Def_Int_TD': 'Int_TD',
        'PFR_Def_PD': 'PD',
        'PFR_Def_Sk': 'Sk',
        'PFR_Def_Comb': 'Comb',
        'PFR_Def_Solo': 'Solo',
        'PFR_Def_Ast': 'Ast',
        'PFR_Def_TFL': 'TFL',
        'PFR_Def_QBHits': 'QBHits',
        'PFR_Def_FR': 'FR',
        'PFR_Def_FF': 'FF',
        'PFR_Def_Sfty': 'Sfty'
    }

    df = df.rename(columns={k: v for k, v in rename_map.items() if k in df.columns})

    # Convert all stat columns to numeric
    numeric_cols = [
        'Year', 'Rank', 'Votes', 'G', 'GS', 'Age',
        'Int', 'Int_Yds', 'Int_TD', 'PD', 'Sk', 'Comb', 'Solo', 'Ast',
        'TFL', 'QBHits', 'FR', 'FF', 'Sfty'
    ]

    for col in numeric_cols:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce')

    # Reorder columns logically
    preferred_order = [
        'Year', 'Rank', 'Player', 'Team', 'Position', 'Votes', 'Age',
        'G', 'GS',
        'Comb', 'Solo', 'Ast', 'TFL',
        'Sk', 'QBHits',
        'Int', 'Int_Yds', 'Int_TD', 'PD',
        'FF', 'FR', 'Sfty'
    ]

    final_columns = [c for c in preferred_order if c in df.columns]
    # Add any remaining columns not in preferred order
    final_columns.extend([c for c in df.columns if c not in final_columns])

    df = df[final_columns]

    print(f"Cleaned shape: {df.shape}")
    print(f"Final columns: {df.columns.tolist()}")

    return df

# Apply final cleanup
dpoy_df_final = final_cleanup_dpoy(dpoy_df_adv)

# Save cleaned version
dpoy_df_final.to_csv('dpoy_df_final.csv', index=False)
files.download('dpoy_df_final.csv')

print("\n" + "="*60)
print("FINAL DPOY DATA SAMPLE:")
print("="*60)
print(dpoy_df_final.head(10))

print("\n" + "="*60)
print("SUMMARY STATISTICS:")
print("="*60)
print(f"Total rows: {len(dpoy_df_final)}")
print(f"Years covered: {dpoy_df_final['Year'].min():.0f} - {dpoy_df_final['Year'].max():.0f}")
print(f"Unique players: {dpoy_df_final['Player'].nunique()}")
print(f"\nMissing values by column:")
print(dpoy_df_final.isnull().sum()[dpoy_df_final.isnull().sum() > 0])

Starting final DPOY cleanup...
Original shape: (331, 34)
Cleaned shape: (331, 22)
Final columns: ['Year', 'Rank', 'Player', 'Team', 'Position', 'Votes', 'Age', 'G', 'GS', 'Comb', 'Solo', 'Ast', 'TFL', 'Sk', 'QBHits', 'Int', 'Int_Yds', 'Int_TD', 'PD', 'FF', 'FR', 'Sfty']


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


FINAL DPOY DATA SAMPLE:
   Year  Rank              Player                 Team Position  Votes   Age  \
0  2024     1  Patrick Surtain II       Denver Broncos       CB   76.0  24.0   
1  2024     2    Trey Hendrickson   Cincinnati Bengals       DE   48.0  30.0   
2  2024     3       Myles Garrett     Cleveland Browns       DE   37.0  29.0   
3  2024     4           T.J. Watt  Pittsburgh Steelers       LB   26.0  30.0   
4  2024     5           Zack Baun  Philadelphia Eagles       LB   15.0  28.0   
5  2024     6        Kerby Joseph        Detroit Lions        S   14.0  24.0   
6  2024     7   Andrew Van Ginkel    Minnesota Vikings      OLB    5.0  29.0   
7  2024     8     Xavier McKinney    Green Bay Packers        S    4.0  26.0   
8  2024     9         Nik Bonitto       Denver Broncos      OLB    4.0  25.0   
9  2024    10       Kyle Hamilton     Baltimore Ravens        S    4.0  23.0   

      G    GS   Comb  ...   TFL    Sk  QBHits  Int  Int_Yds  Int_TD    PD  \
0  16.0  16.0   4

In [ ]:
# OFFENSIVE ROOKIE OF THE YEAR (OROY) SCRAPING AND CLEANING

# Step 1: Scrape OROY data
print("="*60)
print("SCRAPING OROY DATA")
print("="*60)

year = 2024
oroy_dfs = []

while year >= 1980:
    url = f'https://www.pro-football-reference.com/awards/awards_{year}.htm'
    try:
        # Get all tables from the page
        tables = pd.read_html(url)

        # tables[3] is typically OROY voting
        oroy_table = tables[3]

        # Check if columns are multi-level and handle accordingly
        if isinstance(oroy_table.columns, pd.MultiIndex):
            oroy_table.columns = [' '.join(str(col).strip() for col in multi_col if str(col) != 'nan')
                                   for multi_col in oroy_table.columns]
            oroy_table['Year'] = year
        else:
            oroy_table['Year'] = year

        oroy_dfs.append(oroy_table)
        print(f"Scraped OROY {year} - Shape: {oroy_table.shape}")

    except Exception as e:
        print(f"Failed to scrape OROY {year}: {e}")

    time.sleep(3)
    year -= 1

# Combine all dataframes
oroy_df = pd.concat(oroy_dfs, ignore_index=True)
print(f"\nOROY Total rows: {len(oroy_df)}")
print(f"Years covered: {oroy_df['Year'].min()} - {oroy_df['Year'].max()}")

# Save raw scraped data
oroy_df.to_csv('oroy_df.csv', index=False)
files.download('oroy_df.csv')

print("\nSample OROY data:")
print(oroy_df.head(10))

SCRAPING OROY DATA
Scraped OROY 2024 - Shape: (10, 21)
Scraped OROY 2023 - Shape: (8, 21)
Scraped OROY 2022 - Shape: (10, 21)
Scraped OROY 2021 - Shape: (4, 20)
Scraped OROY 2020 - Shape: (2, 20)
Scraped OROY 2019 - Shape: (4, 20)
Scraped OROY 2018 - Shape: (3, 20)
Scraped OROY 2017 - Shape: (3, 20)
Scraped OROY 2016 - Shape: (2, 20)
Scraped OROY 2015 - Shape: (5, 20)
Scraped OROY 2014 - Shape: (3, 20)
Scraped OROY 2013 - Shape: (5, 20)
Scraped OROY 2012 - Shape: (3, 20)
Scraped OROY 2011 - Shape: (2, 20)
Scraped OROY 2010 - Shape: (3, 20)
Scraped OROY 2009 - Shape: (5, 20)
Scraped OROY 2008 - Shape: (4, 20)
Scraped OROY 2007 - Shape: (2, 20)
Scraped OROY 2006 - Shape: (5, 20)
Scraped OROY 2005 - Shape: (4, 20)
Scraped OROY 2004 - Shape: (1, 20)
Scraped OROY 2003 - Shape: (1, 20)
Scraped OROY 2002 - Shape: (4, 20)
Scraped OROY 2001 - Shape: (5, 20)
Scraped OROY 2000 - Shape: (2, 20)
Scraped OROY 1999 - Shape: (2, 20)
Scraped OROY 1998 - Shape: (3, 20)
Scraped OROY 1997 - Shape: (5, 20)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


Sample OROY data:
   Unnamed: 0_level_0 Rk Unnamed: 1_level_0 Pos Unnamed: 2_level_0 Player  \
0                      1                     QB            Jayden Daniels   
1                      2                     TE              Brock Bowers   
2                      3                     QB                    Bo Nix   
3                      4                     WR              Brian Thomas   
4                      5                     WR              Malik Nabers   
5                      6                     RB              Bucky Irving   
6                      7                     OT                   Joe Alt   
7                      8                     QB                Drake Maye   
8                      9                     WR             Ladd McConkey   
9                     10                     QB            Caleb Williams   

   Unnamed: 3_level_0 Tm  Unnamed: 4_level_0 Vote Pts  \
0  Washington Commanders                        485.0   
1      Las Vegas Ra

In [ ]:
# Step 2: Clean OROY data
def clean_oroy_data(oroy_df):
    """
    Clean the scraped OROY DataFrame.
    Similar to MVP/OPOY/DPOY cleaning but for offensive rookies.
    """
    print("Starting OROY data cleaning...")
    print(f"Original shape: {oroy_df.shape}")

    oroy_df_clean = oroy_df.copy()

    # Drop unnecessary columns
    oroy_df_clean = oroy_df_clean.drop(columns=["Unnamed: 0"], errors="ignore")

    # Rename columns
    column_mapping = {
        "Unnamed: 0_level_0 Rk": "Rank",
        "Unnamed: 1_level_0 Pos": "Position",
        "Unnamed: 2_level_0 Player": "Player",
        "Unnamed: 3_level_0 Tm": "Team",
        "Unnamed: 4_level_0 Vote Pts": "VotePoints_1",
        "Unnamed: 4_level_0 Votes": "VotePoints_2",
        "Unnamed: 5_level_0 1st Place": "FirstPlaceVotes",
        "Unnamed: 6_level_0 Share": "VoteShare_1",
        "Unnamed: 5_level_0 Share": "VoteShare_2",
        "Games G": "G",
        "Games GS": "GS",
        # Passing stats
        "Passing Cmp": "Pass_Cmp",
        "Passing Att": "Pass_Att",
        "Passing Yds": "Pass_Yds",
        "Passing TD": "Pass_TD",
        "Passing Int": "Pass_Int",
        # Rushing stats
        "Rushing Att": "Rush_Att",
        "Rushing Yds": "Rush_Yds",
        "Rushing TD": "Rush_TD",
        # Receiving stats
        "Receiving Rec": "Rec",
        "Receiving Yds": "Rec_Yds",
        "Receiving TD": "Rec_TD",
    }

    oroy_df_clean = oroy_df_clean.rename(columns=column_mapping)

    # Consolidate voting columns
    vote_points_cols = [c for c in ["VotePoints_1", "VotePoints_2"] if c in oroy_df_clean.columns]
    if vote_points_cols:
        oroy_df_clean["VotePoints"] = pd.to_numeric(oroy_df_clean[vote_points_cols[0]], errors="coerce")
        for col in vote_points_cols[1:]:
            oroy_df_clean["VotePoints"] = oroy_df_clean["VotePoints"].fillna(
                pd.to_numeric(oroy_df_clean[col], errors="coerce")
            )
        oroy_df_clean = oroy_df_clean.drop(columns=vote_points_cols)

    # Convert FirstPlaceVotes to numeric
    if "FirstPlaceVotes" in oroy_df_clean.columns:
        oroy_df_clean["FirstPlaceVotes"] = pd.to_numeric(oroy_df_clean["FirstPlaceVotes"], errors="coerce")

    # Create unified Votes column (same logic as other awards)
    if "VotePoints" in oroy_df_clean.columns and "Year" in oroy_df_clean.columns:
        votes_2022_plus = oroy_df_clean["VotePoints"] / 4.2
        votes_2022_plus = np.where(pd.isna(votes_2022_plus), np.nan, np.round(votes_2022_plus))
        votes_2022_plus = np.where(
            (votes_2022_plus == 0) & (oroy_df_clean["VotePoints"] > 0),
            1,
            votes_2022_plus
        )

        oroy_df_clean["Votes"] = np.where(
            oroy_df_clean["Year"] >= 2022,
            votes_2022_plus,
            oroy_df_clean["VotePoints"]
        )

    # Convert stat columns to numeric
    numeric_columns = [
        "Rank", "VotePoints", "FirstPlaceVotes", "Votes",
        "G", "GS", "Year",
        "Pass_Cmp", "Pass_Att", "Pass_Yds", "Pass_TD", "Pass_Int",
        "Rush_Att", "Rush_Yds", "Rush_TD",
        "Rec", "Rec_Yds", "Rec_TD"
    ]

    for col in numeric_columns:
        if col in oroy_df_clean.columns:
            oroy_df_clean[col] = pd.to_numeric(oroy_df_clean[col], errors="coerce")

    # Clean text columns
    for col in ["Position", "Player", "Team"]:
        if col in oroy_df_clean.columns:
            oroy_df_clean[col] = oroy_df_clean[col].astype(str).str.strip()

    # Drop unwanted columns
    cols_to_drop = ["VoteShare_1", "VoteShare_2", "VotePoints", "FirstPlaceVotes"]
    oroy_df_clean = oroy_df_clean.drop(
        columns=[c for c in cols_to_drop if c in oroy_df_clean.columns],
        errors="ignore"
    )

    # Reorder columns
    preferred_order = [
        "Year", "Rank", "Player", "Team", "Position", "Votes",
        "G", "GS",
        "Pass_Cmp", "Pass_Att", "Pass_Yds", "Pass_TD", "Pass_Int",
        "Rush_Att", "Rush_Yds", "Rush_TD",
        "Rec", "Rec_Yds", "Rec_TD"
    ]

    final_columns = [c for c in preferred_order if c in oroy_df_clean.columns]
    final_columns.extend([c for c in oroy_df_clean.columns if c not in final_columns])
    oroy_df_clean = oroy_df_clean[final_columns]

    print(f"Cleaned shape: {oroy_df_clean.shape}")
    print(f"Columns: {oroy_df_clean.columns.tolist()}")

    return oroy_df_clean

# Apply cleaning
oroy_df_clean = clean_oroy_data(oroy_df)

# Save cleaned data
oroy_df_clean.to_csv('oroy_df_clean.csv', index=False)
files.download('oroy_df_clean.csv')

print("\nSample cleaned OROY data:")
print(oroy_df_clean.head())

Starting OROY data cleaning...
Original shape: (185, 23)
Cleaned shape: (185, 19)
Columns: ['Year', 'Rank', 'Player', 'Team', 'Position', 'Votes', 'G', 'GS', 'Pass_Cmp', 'Pass_Att', 'Pass_Yds', 'Pass_TD', 'Pass_Int', 'Rush_Att', 'Rush_Yds', 'Rush_TD', 'Rec', 'Rec_Yds', 'Rec_TD']


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


Sample cleaned OROY data:
   Year  Rank          Player                   Team Position  Votes   G  GS  \
0  2024     1  Jayden Daniels  Washington Commanders       QB  115.0  17  17   
1  2024     2    Brock Bowers      Las Vegas Raiders       TE   50.0  17  16   
2  2024     3          Bo Nix         Denver Broncos       QB   36.0  17  17   
3  2024     4    Brian Thomas   Jacksonville Jaguars       WR   19.0  17  16   
4  2024     5    Malik Nabers        New York Giants       WR   12.0  15  13   

   Pass_Cmp  Pass_Att  Pass_Yds  Pass_TD  Pass_Int  Rush_Att  Rush_Yds  \
0       331       480      3568       25         9       148       891   
1         0         0         0        0         0         5        13   
2       376       567      3775       29        12        92       430   
3         0         0         0        0         0         6        48   
4         0         1         0        0         0         5         2   

   Rush_TD  Rec  Rec_Yds  Rec_TD  
0        6  

In [ ]:
# Step 3: Merge with PFR offensive stats
# Note: We'll use the same helper functions from DPOY, plus fetch functions for passing/rushing/receiving

def fetch_passing(year: int, max_retries: int = 3) -> Optional[pd.DataFrame]:
    """Fetch passing stats from Pro Football Reference."""
    url = f'https://www.pro-football-reference.com/years/{year}/passing.htm'

    for attempt in range(max_retries):
        try:
            tables = pd.read_html(url)
            t = _pick_main_table(tables)
            if t is None:
                print(f"No suitable passing table found for {year}")
                return None

            t = _flatten_cols(t)
            print(f" Passing columns for {year}: {t.columns.tolist()}")

            # Standardize column names
            column_map = {}
            for col in t.columns:
                col_str = str(col)
                if col_str == 'Player':
                    column_map[col] = 'Player'
                elif col_str in ['Team', 'Tm']:
                    column_map[col] = 'Tm'
                elif col_str == 'Age':
                    column_map[col] = 'Age'
                elif col_str in ['Pos', 'Position']:
                    column_map[col] = 'Pos'
                elif col_str == 'G':
                    column_map[col] = 'G'
                elif col_str == 'GS':
                    column_map[col] = 'GS'
                elif col_str == 'Passing Cmp' or col_str == 'Cmp':
                    column_map[col] = 'Pass_Cmp'
                elif col_str == 'Passing Att' or col_str == 'Att':
                    column_map[col] = 'Pass_Att'
                elif col_str == 'Passing Yds' or (col_str == 'Yds' and 'Passing' in str(col)):
                    column_map[col] = 'Pass_Yds'
                elif col_str == 'Passing TD' or (col_str == 'TD' and 'Passing' in str(col)):
                    column_map[col] = 'Pass_TD'
                elif col_str == 'Passing Int' or col_str == 'Int':
                    column_map[col] = 'Pass_Int'
                elif col_str == 'Passing Rate' or col_str == 'Rate':
                    column_map[col] = 'Rate'
                elif col_str == 'Passing Y/A' or col_str == 'Y/A':
                    column_map[col] = 'Y/A'
                elif col_str == 'Passing AY/A' or col_str == 'AY/A':
                    column_map[col] = 'AY/A'

            if 'Player' not in column_map.values():
                print(f"WARNING: No Player column found for passing {year}")
                return None

            cols_to_keep = list(column_map.keys())
            t = t[cols_to_keep].copy()
            t = t.rename(columns=column_map)

            if t.columns.duplicated().any():
                t = t.loc[:, ~t.columns.duplicated()]

            t['Year'] = year
            t['Player'] = t['Player'].apply(_clean_player_name)
            t = _prefer_totals_rows(t)

            # Add prefix
            rename_map = {}
            for col in t.columns:
                if col not in ['Player', 'Year']:
                    rename_map[col] = f'PFR_{col}'
            t = t.rename(columns=rename_map)

            print(f" Final passing columns: {t.columns.tolist()}")
            return t

        except Exception as e:
            print(f"Passing fetch attempt {attempt + 1}/{max_retries} failed for {year}: {e}")
            if attempt < max_retries - 1:
                time.sleep(2)
            else:
                return None

def fetch_rushing(year: int, max_retries: int = 3) -> Optional[pd.DataFrame]:
    """Fetch rushing stats from Pro Football Reference."""
    url = f'https://www.pro-football-reference.com/years/{year}/rushing.htm'

    for attempt in range(max_retries):
        try:
            tables = pd.read_html(url)
            t = _pick_main_table(tables)
            if t is None:
                print(f"No suitable rushing table found for {year}")
                return None

            t = _flatten_cols(t)

            column_map = {}
            for col in t.columns:
                col_str = str(col)
                if col_str == 'Player':
                    column_map[col] = 'Player'
                elif col_str in ['Team', 'Tm']:
                    column_map[col] = 'Tm'
                elif col_str == 'Age':
                    column_map[col] = 'Age'
                elif col_str in ['Pos', 'Position']:
                    column_map[col] = 'Pos'
                elif col_str == 'G':
                    column_map[col] = 'G'
                elif col_str == 'GS':
                    column_map[col] = 'GS'
                elif col_str == 'Rushing Att' or col_str == 'Att':
                    column_map[col] = 'Rush_Att'
                elif col_str == 'Rushing Yds' or (col_str == 'Yds' and 'Rushing' in str(col)):
                    column_map[col] = 'Rush_Yds'
                elif col_str == 'Rushing TD' or (col_str == 'TD' and 'Rushing' in str(col)):
                    column_map[col] = 'Rush_TD'
                elif col_str == 'Rushing Y/A' or col_str == 'Y/A':
                    column_map[col] = 'Rush_Y/A'
                elif col_str == 'Rushing Y/G' or col_str == 'Y/G':
                    column_map[col] = 'Rush_Y/G'

            if 'Player' not in column_map.values():
                print(f"WARNING: No Player column found for rushing {year}")
                return None

            cols_to_keep = list(column_map.keys())
            t = t[cols_to_keep].copy()
            t = t.rename(columns=column_map)

            if t.columns.duplicated().any():
                t = t.loc[:, ~t.columns.duplicated()]

            t['Year'] = year
            t['Player'] = t['Player'].apply(_clean_player_name)
            t = _prefer_totals_rows(t)

            rename_map = {}
            for col in t.columns:
                if col not in ['Player', 'Year']:
                    rename_map[col] = f'PFR_{col}'
            t = t.rename(columns=rename_map)

            print(f" Final rushing columns: {t.columns.tolist()}")
            return t

        except Exception as e:
            print(f"Rushing fetch attempt {attempt + 1}/{max_retries} failed for {year}: {e}")
            if attempt < max_retries - 1:
                time.sleep(2)
            else:
                return None

def fetch_receiving(year: int, max_retries: int = 3) -> Optional[pd.DataFrame]:
    """Fetch receiving stats from Pro Football Reference."""
    url = f'https://www.pro-football-reference.com/years/{year}/receiving.htm'

    for attempt in range(max_retries):
        try:
            tables = pd.read_html(url)
            t = _pick_main_table(tables)
            if t is None:
                print(f"No suitable receiving table found for {year}")
                return None

            t = _flatten_cols(t)

            column_map = {}
            for col in t.columns:
                col_str = str(col)
                if col_str == 'Player':
                    column_map[col] = 'Player'
                elif col_str in ['Team', 'Tm']:
                    column_map[col] = 'Tm'
                elif col_str == 'Age':
                    column_map[col] = 'Age'
                elif col_str in ['Pos', 'Position']:
                    column_map[col] = 'Pos'
                elif col_str == 'G':
                    column_map[col] = 'G'
                elif col_str == 'GS':
                    column_map[col] = 'GS'
                elif col_str == 'Receiving Rec' or col_str == 'Rec':
                    column_map[col] = 'Rec'
                elif col_str == 'Receiving Yds' or (col_str == 'Yds' and 'Receiving' in str(col)):
                    column_map[col] = 'Rec_Yds'
                elif col_str == 'Receiving TD' or (col_str == 'TD' and 'Receiving' in str(col)):
                    column_map[col] = 'Rec_TD'
                elif col_str == 'Receiving Tgt' or col_str == 'Tgt':
                    column_map[col] = 'Tgt'
                elif col_str == 'Receiving Y/R' or col_str == 'Y/R':
                    column_map[col] = 'Rec_Y/R'
                elif col_str == 'Receiving Y/G' or col_str == 'Y/G':
                    column_map[col] = 'Rec_Y/G'

            if 'Player' not in column_map.values():
                print(f"WARNING: No Player column found for receiving {year}")
                return None

            cols_to_keep = list(column_map.keys())
            t = t[cols_to_keep].copy()
            t = t.rename(columns=column_map)

            if t.columns.duplicated().any():
                t = t.loc[:, ~t.columns.duplicated()]

            t['Year'] = year
            t['Player'] = t['Player'].apply(_clean_player_name)
            t = _prefer_totals_rows(t)

            rename_map = {}
            for col in t.columns:
                if col not in ['Player', 'Year']:
                    rename_map[col] = f'PFR_{col}'
            t = t.rename(columns=rename_map)

            print(f" Final receiving columns: {t.columns.tolist()}")
            return t

        except Exception as e:
            print(f"Receiving fetch attempt {attempt + 1}/{max_retries} failed for {year}: {e}")
            if attempt < max_retries - 1:
                time.sleep(2)
            else:
                return None

# Merge OROY with PFR offensive stats
def merge_oroy_with_pfr(oroy_df_clean: pd.DataFrame, sleep_time: float = 3.0, debug: bool = True) -> pd.DataFrame:
    """Merge OROY data with PFR passing, rushing, and receiving stats."""

    if 'Player' not in oroy_df_clean.columns or 'Year' not in oroy_df_clean.columns:
        raise ValueError("oroy_df_clean must contain 'Player' and 'Year' columns")

    oroy_df_aug = oroy_df_clean.copy()
    oroy_df_aug['Player_clean'] = oroy_df_aug['Player'].apply(_clean_player_name)

    years = sorted(oroy_df_aug['Year'].dropna().astype(int).unique().tolist(), reverse=True)
    print(f"Processing years: {years}")

    pass_frames = []
    rush_frames = []
    rec_frames = []

    for i, year in enumerate(years):
        print(f"\n{'='*50}")
        print(f"Processing year {year} ({i+1}/{len(years)})...")

        year_oroy_players = set(oroy_df_aug[oroy_df_aug['Year'] == year]['Player_clean'].dropna())
        print(f"OROY players in {year}: {sorted(year_oroy_players)}")

        # Fetch passing data
        pass_data = fetch_passing(year)
        if pass_data is not None:
            pass_data['Player_clean'] = pass_data['Player']
            pass_frames.append(pass_data)
            print(f" Passing data: {pass_data.shape[0]} players, {pass_data.shape[1]} columns")

        # Fetch rushing data
        rush_data = fetch_rushing(year)
        if rush_data is not None:
            rush_data['Player_clean'] = rush_data['Player']
            rush_frames.append(rush_data)
            print(f" Rushing data: {rush_data.shape[0]} players, {rush_data.shape[1]} columns")

        # Fetch receiving data
        rec_data = fetch_receiving(year)
        if rec_data is not None:
            rec_data['Player_clean'] = rec_data['Player']
            rec_frames.append(rec_data)
            print(f" Receiving data: {rec_data.shape[0]} players, {rec_data.shape[1]} columns")

        time.sleep(sleep_time)

    # Combine all data
    pass_all = pd.concat(pass_frames, ignore_index=True) if pass_frames else pd.DataFrame()
    rush_all = pd.concat(rush_frames, ignore_index=True) if rush_frames else pd.DataFrame()
    rec_all = pd.concat(rec_frames, ignore_index=True) if rec_frames else pd.DataFrame()

    print(f"\n{'='*50}")
    print(f"SUMMARY:")
    print(f"Combined passing data: {pass_all.shape}")
    print(f"Combined rushing data: {rush_all.shape}")
    print(f"Combined receiving data: {rec_all.shape}")

    # Merge with OROY data
    merged = oroy_df_aug.copy()

    if not pass_all.empty:
        print(f"\nMerging passing data...")
        merged = merged.merge(
            pass_all.drop(columns=['Player'], errors='ignore'),
            on=['Year', 'Player_clean'],
            how='left',
            suffixes=('', '_pass_dup')
        )

    if not rush_all.empty:
        print(f"Merging rushing data...")
        merged = merged.merge(
            rush_all.drop(columns=['Player'], errors='ignore'),
            on=['Year', 'Player_clean'],
            how='left',
            suffixes=('', '_rush_dup')
        )

    if not rec_all.empty:
        print(f"Merging receiving data...")
        merged = merged.merge(
            rec_all.drop(columns=['Player'], errors='ignore'),
            on=['Year', 'Player_clean'],
            how='left',
            suffixes=('', '_rec_dup')
        )

    merged = merged.drop(columns=['Player_clean'])

    # Convert numeric columns
    for col in merged.columns:
        if 'PFR_' in col:
            merged[col] = pd.to_numeric(merged[col], errors='ignore')

    print(f"\nFinal merged shape: {merged.shape}")
    return merged

# Execute the merge
oroy_df_adv = merge_oroy_with_pfr(oroy_df_clean, sleep_time=3, debug=False)

# Save
oroy_df_adv.to_csv('oroy_df_adv.csv', index=False)
files.download('oroy_df_adv.csv')

print("\nFirst few rows of merged OROY data:")
print(oroy_df_adv.head())

NameError: name '_clean_player_name' is not defined

In [ ]:
# Step 4: Final cleanup of merged OROY data
def final_cleanup_oroy(oroy_df_adv):
    """
    Final cleanup of merged OROY dataframe to remove redundant columns
    and consolidate overlapping data.
    """
    print("Starting final OROY cleanup...")
    print(f"Original shape: {oroy_df_adv.shape}")

    df = oroy_df_adv.copy()

    # Drop redundant columns from original OROY scrape
    columns_to_drop = [
        'Pass_Cmp', 'Pass_Att', 'Pass_Yds', 'Pass_TD', 'Pass_Int',
        'Rush_Att', 'Rush_Yds', 'Rush_TD',
        'Rec', 'Rec_Yds', 'Rec_TD'
    ]

    # Only drop if PFR versions exist
    for col in columns_to_drop:
        pfr_col = f'PFR_{col}'
        if pfr_col in df.columns and col in df.columns:
            df = df.drop(columns=[col], errors='ignore')

    # Consolidate G and GS (prefer PFR version if available)
    for stat in ['G', 'GS']:
        pfr_cols = [c for c in df.columns if c.startswith('PFR_') and c.endswith(f'_{stat}')]
        if pfr_cols and stat in df.columns:
            # Use first available PFR version
            df[stat] = df[pfr_cols[0]].fillna(df[stat])
            df = df.drop(columns=pfr_cols, errors='ignore')
        elif pfr_cols:
            df = df.rename(columns={pfr_cols[0]: stat})
            df = df.drop(columns=pfr_cols[1:], errors='ignore')

    # Consolidate Team and Position (prefer original OROY version)
    for tm_col in [c for c in df.columns if c.startswith('PFR_') and 'Tm' in c]:
        if 'Team' in df.columns:
            df['Team'] = df['Team'].fillna(df[tm_col])
        df = df.drop(columns=[tm_col], errors='ignore')

    for pos_col in [c for c in df.columns if c.startswith('PFR_') and 'Pos' in c]:
        if 'Position' in df.columns:
            df['Position'] = df['Position'].fillna(df[pos_col])
        df = df.drop(columns=[pos_col], errors='ignore')

    # Rename PFR columns to cleaner names
    rename_map = {}
    for col in df.columns:
        if col.startswith('PFR_'):
            # Remove PFR_ prefix
            new_name = col.replace('PFR_', '')
            rename_map[col] = new_name

    df = df.rename(columns=rename_map)

    # Convert all stat columns to numeric
    numeric_cols = [
        'Year', 'Rank', 'Votes', 'G', 'GS', 'Age',
        'Pass_Cmp', 'Pass_Att', 'Pass_Yds', 'Pass_TD', 'Pass_Int', 'Rate', 'Y/A', 'AY/A',
        'Rush_Att', 'Rush_Yds', 'Rush_TD', 'Rush_Y/A', 'Rush_Y/G',
        'Rec', 'Rec_Yds', 'Rec_TD', 'Tgt', 'Rec_Y/R', 'Rec_Y/G'
    ]

    for col in numeric_cols:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce')

    # Reorder columns logically
    preferred_order = [
        'Year', 'Rank', 'Player', 'Team', 'Position', 'Votes', 'Age',
        'G', 'GS',
        # Passing
        'Pass_Cmp', 'Pass_Att', 'Pass_Yds', 'Pass_TD', 'Pass_Int', 'Rate', 'Y/A', 'AY/A',
        # Rushing
        'Rush_Att', 'Rush_Yds', 'Rush_TD', 'Rush_Y/A', 'Rush_Y/G',
        # Receiving
        'Rec', 'Tgt', 'Rec_Yds', 'Rec_TD', 'Rec_Y/R', 'Rec_Y/G'
    ]

    final_columns = [c for c in preferred_order if c in df.columns]
    # Add any remaining columns not in preferred order
    final_columns.extend([c for c in df.columns if c not in final_columns])

    df = df[final_columns]

    print(f"Cleaned shape: {df.shape}")
    print(f"Final columns: {df.columns.tolist()}")

    return df

# Apply final cleanup
oroy_df_final = final_cleanup_oroy(oroy_df_adv)

# Save cleaned version
oroy_df_final.to_csv('oroy_df_final.csv', index=False)
files.download('oroy_df_final.csv')

print("\n" + "="*60)
print("FINAL OROY DATA SAMPLE:")
print("="*60)
print(oroy_df_final.head(10))

print("\n" + "="*60)
print("SUMMARY STATISTICS:")
print("="*60)
print(f"Total rows: {len(oroy_df_final)}")
print(f"Years covered: {oroy_df_final['Year'].min():.0f} - {oroy_df_final['Year'].max():.0f}")
print(f"Unique players: {oroy_df_final['Player'].nunique()}")
print(f"\nMissing values by column:")
print(oroy_df_final.isnull().sum()[oroy_df_final.isnull().sum() > 0])

In [ ]:
# OFFENSIVE ROOKIE OF THE YEAR (OROY) SCRAPING AND CLEANING
# Complete code with all helper functions

import pandas as pd
import numpy as np
import time
from typing import Optional

# ============================================================================
# HELPER FUNCTIONS (from DPOY code)
# ============================================================================

def _clean_player_name(name: str) -> str:
    """Remove asterisks and plus signs from player names."""
    if pd.isna(name):
        return name
    name = str(name).strip()
    name = name.replace('*', '').replace('+', '')
    return name.strip()

def _pick_main_table(tables: list) -> Optional[pd.DataFrame]:
    """Pick the largest table from a list of tables."""
    if not tables:
        return None
    largest = max(tables, key=lambda t: t.shape[0] * t.shape[1])
    return largest

def _flatten_cols(df: pd.DataFrame) -> pd.DataFrame:
    """Flatten multi-level columns to single level."""
    if isinstance(df.columns, pd.MultiIndex):
        df.columns = [' '.join(str(c).strip() for c in col if str(c) != 'nan').strip()
                      for col in df.columns]
    return df

def _prefer_totals_rows(df: pd.DataFrame) -> pd.DataFrame:
    """
    For players with multiple rows (multiple teams), prefer the row with Tm='TOT' if available.
    Otherwise, keep the first row.
    """
    if 'Tm' not in df.columns or 'Player' not in df.columns:
        return df

    df = df.copy()

    # Group by Player and Year
    if 'Year' in df.columns:
        groups = df.groupby(['Player', 'Year'])
    else:
        groups = df.groupby('Player')

    result_rows = []
    for name, group in groups:
        if len(group) == 1:
            result_rows.append(group.iloc[0])
        else:
            # Check for TOT row
            tot_rows = group[group['Tm'] == 'TOT']
            if not tot_rows.empty:
                result_rows.append(tot_rows.iloc[0])
            else:
                result_rows.append(group.iloc[0])

    return pd.DataFrame(result_rows).reset_index(drop=True)

# ============================================================================
# STEP 1: SCRAPE OROY DATA
# ============================================================================

def scrape_oroy_data(start_year=2024, end_year=1980):
    """Scrape OROY voting data from Pro Football Reference."""
    print("="*60)
    print("SCRAPING OROY DATA")
    print("="*60)

    year = start_year
    oroy_dfs = []

    while year >= end_year:
        url = f'https://www.pro-football-reference.com/awards/awards_{year}.htm'
        try:
            tables = pd.read_html(url)
            oroy_table = tables[3]  # OROY is typically table index 3

            if isinstance(oroy_table.columns, pd.MultiIndex):
                oroy_table.columns = [' '.join(str(col).strip() for col in multi_col if str(col) != 'nan')
                                       for multi_col in oroy_table.columns]

            oroy_table['Year'] = year
            oroy_dfs.append(oroy_table)
            print(f"Scraped OROY {year} - Shape: {oroy_table.shape}")

        except Exception as e:
            print(f"Failed to scrape OROY {year}: {e}")

        time.sleep(3)
        year -= 1

    oroy_df = pd.concat(oroy_dfs, ignore_index=True)
    print(f"\nOROY Total rows: {len(oroy_df)}")
    print(f"Years covered: {oroy_df['Year'].min()} - {oroy_df['Year'].max()}")

    return oroy_df

# ============================================================================
# STEP 2: CLEAN OROY DATA
# ============================================================================

def clean_oroy_data(oroy_df):
    """Clean the scraped OROY DataFrame."""
    print("\nStarting OROY data cleaning...")
    print(f"Original shape: {oroy_df.shape}")

    oroy_df_clean = oroy_df.copy()

    # Drop unnecessary columns
    oroy_df_clean = oroy_df_clean.drop(columns=["Unnamed: 0"], errors="ignore")

    # Rename columns
    column_mapping = {
        "Unnamed: 0_level_0 Rk": "Rank",
        "Unnamed: 1_level_0 Pos": "Position",
        "Unnamed: 2_level_0 Player": "Player",
        "Unnamed: 3_level_0 Tm": "Team",
        "Unnamed: 4_level_0 Vote Pts": "VotePoints_1",
        "Unnamed: 4_level_0 Votes": "VotePoints_2",
        "Unnamed: 5_level_0 1st Place": "FirstPlaceVotes",
        "Unnamed: 6_level_0 Share": "VoteShare_1",
        "Unnamed: 5_level_0 Share": "VoteShare_2",
        "Games G": "G",
        "Games GS": "GS",
        "Passing Cmp": "Pass_Cmp",
        "Passing Att": "Pass_Att",
        "Passing Yds": "Pass_Yds",
        "Passing TD": "Pass_TD",
        "Passing Int": "Pass_Int",
        "Rushing Att": "Rush_Att",
        "Rushing Yds": "Rush_Yds",
        "Rushing TD": "Rush_TD",
        "Receiving Rec": "Rec",
        "Receiving Yds": "Rec_Yds",
        "Receiving TD": "Rec_TD",
    }

    oroy_df_clean = oroy_df_clean.rename(columns=column_mapping)

    # Consolidate voting columns
    vote_points_cols = [c for c in ["VotePoints_1", "VotePoints_2"] if c in oroy_df_clean.columns]
    if vote_points_cols:
        oroy_df_clean["VotePoints"] = pd.to_numeric(oroy_df_clean[vote_points_cols[0]], errors="coerce")
        for col in vote_points_cols[1:]:
            oroy_df_clean["VotePoints"] = oroy_df_clean["VotePoints"].fillna(
                pd.to_numeric(oroy_df_clean[col], errors="coerce")
            )
        oroy_df_clean = oroy_df_clean.drop(columns=vote_points_cols)

    # Convert FirstPlaceVotes to numeric
    if "FirstPlaceVotes" in oroy_df_clean.columns:
        oroy_df_clean["FirstPlaceVotes"] = pd.to_numeric(oroy_df_clean["FirstPlaceVotes"], errors="coerce")

    # Create unified Votes column
    if "VotePoints" in oroy_df_clean.columns and "Year" in oroy_df_clean.columns:
        votes_2022_plus = oroy_df_clean["VotePoints"] / 4.2
        votes_2022_plus = np.where(pd.isna(votes_2022_plus), np.nan, np.round(votes_2022_plus))
        votes_2022_plus = np.where(
            (votes_2022_plus == 0) & (oroy_df_clean["VotePoints"] > 0),
            1,
            votes_2022_plus
        )

        oroy_df_clean["Votes"] = np.where(
            oroy_df_clean["Year"] >= 2022,
            votes_2022_plus,
            oroy_df_clean["VotePoints"]
        )

    # Convert stat columns to numeric
    numeric_columns = [
        "Rank", "VotePoints", "FirstPlaceVotes", "Votes",
        "G", "GS", "Year",
        "Pass_Cmp", "Pass_Att", "Pass_Yds", "Pass_TD", "Pass_Int",
        "Rush_Att", "Rush_Yds", "Rush_TD",
        "Rec", "Rec_Yds", "Rec_TD"
    ]

    for col in numeric_columns:
        if col in oroy_df_clean.columns:
            oroy_df_clean[col] = pd.to_numeric(oroy_df_clean[col], errors="coerce")

    # Clean text columns
    for col in ["Position", "Player", "Team"]:
        if col in oroy_df_clean.columns:
            oroy_df_clean[col] = oroy_df_clean[col].astype(str).str.strip()

    # Drop unwanted columns
    cols_to_drop = ["VoteShare_1", "VoteShare_2", "VotePoints", "FirstPlaceVotes"]
    oroy_df_clean = oroy_df_clean.drop(
        columns=[c for c in cols_to_drop if c in oroy_df_clean.columns],
        errors="ignore"
    )

    # Reorder columns
    preferred_order = [
        "Year", "Rank", "Player", "Team", "Position", "Votes",
        "G", "GS",
        "Pass_Cmp", "Pass_Att", "Pass_Yds", "Pass_TD", "Pass_Int",
        "Rush_Att", "Rush_Yds", "Rush_TD",
        "Rec", "Rec_Yds", "Rec_TD"
    ]

    final_columns = [c for c in preferred_order if c in oroy_df_clean.columns]
    final_columns.extend([c for c in oroy_df_clean.columns if c not in final_columns])
    oroy_df_clean = oroy_df_clean[final_columns]

    print(f"Cleaned shape: {oroy_df_clean.shape}")
    print(f"Columns: {oroy_df_clean.columns.tolist()}")

    return oroy_df_clean

# ============================================================================
# STEP 3: FETCH PFR OFFENSIVE STATS
# ============================================================================

def fetch_passing(year: int, max_retries: int = 3) -> Optional[pd.DataFrame]:
    """Fetch passing stats from Pro Football Reference."""
    url = f'https://www.pro-football-reference.com/years/{year}/passing.htm'

    for attempt in range(max_retries):
        try:
            tables = pd.read_html(url)
            t = _pick_main_table(tables)
            if t is None:
                return None

            t = _flatten_cols(t)

            column_map = {}
            for col in t.columns:
                col_str = str(col)
                if 'Player' in col_str:
                    column_map[col] = 'Player'
                elif col_str in ['Team', 'Tm']:
                    column_map[col] = 'Tm'
                elif col_str == 'Age':
                    column_map[col] = 'Age'
                elif 'Pos' in col_str:
                    column_map[col] = 'Pos'
                elif col_str == 'G':
                    column_map[col] = 'G'
                elif col_str == 'GS':
                    column_map[col] = 'GS'
                elif 'Cmp' in col_str:
                    column_map[col] = 'Pass_Cmp'
                elif 'Att' in col_str and 'Passing' in col_str:
                    column_map[col] = 'Pass_Att'
                elif 'Yds' in col_str and 'Passing' in col_str:
                    column_map[col] = 'Pass_Yds'
                elif 'TD' in col_str and 'Passing' in col_str:
                    column_map[col] = 'Pass_TD'
                elif 'Int' in col_str:
                    column_map[col] = 'Pass_Int'
                elif 'Rate' in col_str:
                    column_map[col] = 'Rate'

            if 'Player' not in column_map.values():
                return None

            cols_to_keep = list(column_map.keys())
            t = t[cols_to_keep].copy()
            t = t.rename(columns=column_map)

            if t.columns.duplicated().any():
                t = t.loc[:, ~t.columns.duplicated()]

            t['Year'] = year
            t['Player'] = t['Player'].apply(_clean_player_name)
            t = _prefer_totals_rows(t)

            rename_map = {col: f'PFR_{col}' for col in t.columns if col not in ['Player', 'Year']}
            t = t.rename(columns=rename_map)

            return t

        except Exception as e:
            if attempt < max_retries - 1:
                time.sleep(2)
            else:
                return None

def fetch_rushing(year: int, max_retries: int = 3) -> Optional[pd.DataFrame]:
    """Fetch rushing stats from Pro Football Reference."""
    url = f'https://www.pro-football-reference.com/years/{year}/rushing.htm'

    for attempt in range(max_retries):
        try:
            tables = pd.read_html(url)
            t = _pick_main_table(tables)
            if t is None:
                return None

            t = _flatten_cols(t)

            column_map = {}
            for col in t.columns:
                col_str = str(col)
                if 'Player' in col_str:
                    column_map[col] = 'Player'
                elif col_str in ['Team', 'Tm']:
                    column_map[col] = 'Tm'
                elif col_str == 'Age':
                    column_map[col] = 'Age'
                elif 'Pos' in col_str:
                    column_map[col] = 'Pos'
                elif col_str == 'G':
                    column_map[col] = 'G'
                elif col_str == 'GS':
                    column_map[col] = 'GS'
                elif 'Att' in col_str and 'Rushing' in col_str:
                    column_map[col] = 'Rush_Att'
                elif 'Yds' in col_str and 'Rushing' in col_str:
                    column_map[col] = 'Rush_Yds'
                elif 'TD' in col_str and 'Rushing' in col_str:
                    column_map[col] = 'Rush_TD'

            if 'Player' not in column_map.values():
                return None

            cols_to_keep = list(column_map.keys())
            t = t[cols_to_keep].copy()
            t = t.rename(columns=column_map)

            if t.columns.duplicated().any():
                t = t.loc[:, ~t.columns.duplicated()]

            t['Year'] = year
            t['Player'] = t['Player'].apply(_clean_player_name)
            t = _prefer_totals_rows(t)

            rename_map = {col: f'PFR_{col}' for col in t.columns if col not in ['Player', 'Year']}
            t = t.rename(columns=rename_map)

            return t

        except Exception as e:
            if attempt < max_retries - 1:
                time.sleep(2)
            else:
                return None

def fetch_receiving(year: int, max_retries: int = 3) -> Optional[pd.DataFrame]:
    """Fetch receiving stats from Pro Football Reference."""
    url = f'https://www.pro-football-reference.com/years/{year}/receiving.htm'

    for attempt in range(max_retries):
        try:
            tables = pd.read_html(url)
            t = _pick_main_table(tables)
            if t is None:
                return None

            t = _flatten_cols(t)

            column_map = {}
            for col in t.columns:
                col_str = str(col)
                if 'Player' in col_str:
                    column_map[col] = 'Player'
                elif col_str in ['Team', 'Tm']:
                    column_map[col] = 'Tm'
                elif col_str == 'Age':
                    column_map[col] = 'Age'
                elif 'Pos' in col_str:
                    column_map[col] = 'Pos'
                elif col_str == 'G':
                    column_map[col] = 'G'
                elif col_str == 'GS':
                    column_map[col] = 'GS'
                elif 'Rec' in col_str and 'Receiving' in col_str:
                    column_map[col] = 'Rec'
                elif 'Yds' in col_str and 'Receiving' in col_str:
                    column_map[col] = 'Rec_Yds'
                elif 'TD' in col_str and 'Receiving' in col_str:
                    column_map[col] = 'Rec_TD'
                elif 'Tgt' in col_str:
                    column_map[col] = 'Tgt'

            if 'Player' not in column_map.values():
                return None

            cols_to_keep = list(column_map.keys())
            t = t[cols_to_keep].copy()
            t = t.rename(columns=column_map)

            if t.columns.duplicated().any():
                t = t.loc[:, ~t.columns.duplicated()]

            t['Year'] = year
            t['Player'] = t['Player'].apply(_clean_player_name)
            t = _prefer_totals_rows(t)

            rename_map = {col: f'PFR_{col}' for col in t.columns if col not in ['Player', 'Year']}
            t = t.rename(columns=rename_map)

            return t

        except Exception as e:
            if attempt < max_retries - 1:
                time.sleep(2)
            else:
                return None

# ============================================================================
# STEP 4: MERGE OROY WITH PFR STATS
# ============================================================================

def merge_oroy_with_pfr(oroy_df_clean: pd.DataFrame, sleep_time: float = 3.0) -> pd.DataFrame:
    """Merge OROY data with PFR passing, rushing, and receiving stats."""

    if 'Player' not in oroy_df_clean.columns or 'Year' not in oroy_df_clean.columns:
        raise ValueError("oroy_df_clean must contain 'Player' and 'Year' columns")

    oroy_df_aug = oroy_df_clean.copy()
    oroy_df_aug['Player_clean'] = oroy_df_aug['Player'].apply(_clean_player_name)

    years = sorted(oroy_df_aug['Year'].dropna().astype(int).unique().tolist(), reverse=True)
    print(f"\nProcessing {len(years)} years: {min(years)} - {max(years)}")

    pass_frames = []
    rush_frames = []
    rec_frames = []

    for i, year in enumerate(years):
        print(f"Year {year} ({i+1}/{len(years)})...", end=' ')

        pass_data = fetch_passing(year)
        if pass_data is not None:
            pass_data['Player_clean'] = pass_data['Player']
            pass_frames.append(pass_data)
            print("✓ Pass", end=' ')

        rush_data = fetch_rushing(year)
        if rush_data is not None:
            rush_data['Player_clean'] = rush_data['Player']
            rush_frames.append(rush_data)
            print("✓ Rush", end=' ')

        rec_data = fetch_receiving(year)
        if rec_data is not None:
            rec_data['Player_clean'] = rec_data['Player']
            rec_frames.append(rec_data)
            print("✓ Rec", end='')

        print()
        time.sleep(sleep_time)

    # Combine all data
    pass_all = pd.concat(pass_frames, ignore_index=True) if pass_frames else pd.DataFrame()
    rush_all = pd.concat(rush_frames, ignore_index=True) if rush_frames else pd.DataFrame()
    rec_all = pd.concat(rec_frames, ignore_index=True) if rec_frames else pd.DataFrame()

    print(f"\nCombined passing: {pass_all.shape}")
    print(f"Combined rushing: {rush_all.shape}")
    print(f"Combined receiving: {rec_all.shape}")

    # Merge with OROY data
    merged = oroy_df_aug.copy()

    if not pass_all.empty:
        merged = merged.merge(
            pass_all.drop(columns=['Player'], errors='ignore'),
            on=['Year', 'Player_clean'],
            how='left',
            suffixes=('', '_pass_dup')
        )

    if not rush_all.empty:
        merged = merged.merge(
            rush_all.drop(columns=['Player'], errors='ignore'),
            on=['Year', 'Player_clean'],
            how='left',
            suffixes=('', '_rush_dup')
        )

    if not rec_all.empty:
        merged = merged.merge(
            rec_all.drop(columns=['Player'], errors='ignore'),
            on=['Year', 'Player_clean'],
            how='left',
            suffixes=('', '_rec_dup')
        )

    merged = merged.drop(columns=['Player_clean'])

    # Convert numeric columns
    for col in merged.columns:
        if 'PFR_' in col:
            merged[col] = pd.to_numeric(merged[col], errors='ignore')

    print(f"\nFinal merged shape: {merged.shape}")
    return merged

# ============================================================================
# MAIN EXECUTION
# ============================================================================

if __name__ == "__main__":
    # Step 1: Scrape
    oroy_df = scrape_oroy_data(start_year=2024, end_year=1980)
    oroy_df.to_csv('oroy_df.csv', index=False)

    # Step 2: Clean
    oroy_df_clean = clean_oroy_data(oroy_df)
    oroy_df_clean.to_csv('oroy_df_clean.csv', index=False)

    # Step 3: Merge with PFR stats
    oroy_df_adv = merge_oroy_with_pfr(oroy_df_clean, sleep_time=3)
    oroy_df_adv.to_csv('oroy_df_adv.csv', index=False)

    print("\n" + "="*60)
    print("COMPLETE!")
    print("="*60)
    print(f"Final dataset shape: {oroy_df_adv.shape}")
    print(f"\nSample data:")
    print(oroy_df_adv.head())

SCRAPING OROY DATA
Scraped OROY 2024 - Shape: (10, 21)
Scraped OROY 2023 - Shape: (8, 21)
Scraped OROY 2022 - Shape: (10, 21)
Scraped OROY 2021 - Shape: (4, 20)
Scraped OROY 2020 - Shape: (2, 20)
Scraped OROY 2019 - Shape: (4, 20)
Scraped OROY 2018 - Shape: (3, 20)
Scraped OROY 2017 - Shape: (3, 20)
Scraped OROY 2016 - Shape: (2, 20)
Scraped OROY 2015 - Shape: (5, 20)
Scraped OROY 2014 - Shape: (3, 20)
Scraped OROY 2013 - Shape: (5, 20)
Scraped OROY 2012 - Shape: (3, 20)
Scraped OROY 2011 - Shape: (2, 20)
Scraped OROY 2010 - Shape: (3, 20)
Scraped OROY 2009 - Shape: (5, 20)
Scraped OROY 2008 - Shape: (4, 20)
Scraped OROY 2007 - Shape: (2, 20)
Scraped OROY 2006 - Shape: (5, 20)
Scraped OROY 2005 - Shape: (4, 20)
Scraped OROY 2004 - Shape: (1, 20)
Scraped OROY 2003 - Shape: (1, 20)
Scraped OROY 2002 - Shape: (4, 20)
Scraped OROY 2001 - Shape: (5, 20)
Scraped OROY 2000 - Shape: (2, 20)
Scraped OROY 1999 - Shape: (2, 20)
Scraped OROY 1998 - Shape: (3, 20)
Scraped OROY 1997 - Shape: (5, 20)

/tmp/ipython-input-1134763699.py:496: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  merged[col] = pd.to_numeric(merged[col], errors='ignore')


In [ ]:
# If running in Google Colab, uncomment these lines to download files:
from google.colab import files
files.download('oroy_df.csv')
files.download('oroy_df_clean.csv')
files.download('oroy_df_adv.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import pandas as pd
import numpy as np

def clean_oroy_advanced(df):
    """
    Clean the OROY advanced dataset by removing duplicate columns
    and consolidating PFR stats with voting stats.
    """
    print("Starting advanced OROY data cleaning...")
    print(f"Original shape: {df.shape}")
    print(f"Original columns: {df.columns.tolist()}")

    df_clean = df.copy()

    # ============================================================================
    # STEP 1: Handle duplicate position columns
    # ============================================================================

    # Consolidate Position columns (prefer non-PFR first, then PFR)
    pos_cols = [c for c in df_clean.columns if 'Pos' in c]
    print(f"\nPosition columns found: {pos_cols}")

    if 'Position' in df_clean.columns:
        # Keep the main Position column
        df_clean['Position'] = df_clean['Position'].fillna(
            df_clean.get('PFR_Pos', df_clean.get('PFR_Pos_rush_dup', df_clean.get('PFR_Pos_rec_dup', '')))
        )
    elif 'PFR_Pos' in df_clean.columns:
        df_clean['Position'] = df_clean['PFR_Pos']

    # Drop all PFR position columns
    pos_cols_to_drop = ['PFR_Pos', 'PFR_Pos_rush_dup', 'PFR_Pos_rec_dup']
    df_clean = df_clean.drop(columns=[c for c in pos_cols_to_drop if c in df_clean.columns])

    # ============================================================================
    # STEP 2: Consolidate team columns
    # ============================================================================

    if 'Team' in df_clean.columns and 'PFR_Tm' in df_clean.columns:
        # Fill missing Team values with PFR_Tm
        df_clean['Team'] = df_clean['Team'].fillna(df_clean['PFR_Tm'])
        df_clean = df_clean.drop(columns=['PFR_Tm'])
    elif 'PFR_Tm' in df_clean.columns:
        df_clean = df_clean.rename(columns={'PFR_Tm': 'Team'})

    # ============================================================================
    # STEP 3: Consolidate games columns (G, GS)
    # ============================================================================

    # Games (G)
    if 'G' in df_clean.columns and 'PFR_G' in df_clean.columns:
        df_clean['G'] = df_clean['G'].fillna(df_clean['PFR_G'])
        df_clean = df_clean.drop(columns=['PFR_G'])
    elif 'PFR_G' in df_clean.columns:
        df_clean = df_clean.rename(columns={'PFR_G': 'G'})

    # Games Started (GS)
    if 'GS' in df_clean.columns and 'PFR_GS' in df_clean.columns:
        df_clean['GS'] = df_clean['GS'].fillna(df_clean['PFR_GS'])
        df_clean = df_clean.drop(columns=['PFR_GS'])
    elif 'PFR_GS' in df_clean.columns:
        df_clean = df_clean.rename(columns={'PFR_GS': 'GS'})

    # ============================================================================
    # STEP 4: Consolidate passing stats
    # ============================================================================

    passing_stats = {
        'Pass_Cmp': 'PFR_Pass_Cmp',
        'Pass_Att': 'PFR_Pass_Att',
        'Pass_Yds': 'PFR_Pass_Yds',
        'Pass_TD': 'PFR_Pass_TD',
        'Pass_Int': 'PFR_Pass_Int'
    }

    for base_col, pfr_col in passing_stats.items():
        if base_col in df_clean.columns and pfr_col in df_clean.columns:
            # Use PFR data if base is 0 or NaN
            df_clean[base_col] = np.where(
                (df_clean[base_col].isna()) | (df_clean[base_col] == 0),
                df_clean[pfr_col],
                df_clean[base_col]
            )
            df_clean = df_clean.drop(columns=[pfr_col])
        elif pfr_col in df_clean.columns:
            df_clean = df_clean.rename(columns={pfr_col: base_col})

    # Keep PFR_Rate (passer rating) as additional stat
    # No consolidation needed - it's a new column

    # ============================================================================
    # STEP 5: Consolidate rushing stats
    # ============================================================================

    rushing_stats = {
        'Rush_Att': 'PFR_Rush_Att',
        'Rush_Yds': 'PFR_Rush_Yds',
        'Rush_TD': 'PFR_Rush_TD'
    }

    for base_col, pfr_col in rushing_stats.items():
        if base_col in df_clean.columns and pfr_col in df_clean.columns:
            df_clean[base_col] = np.where(
                (df_clean[base_col].isna()) | (df_clean[base_col] == 0),
                df_clean[pfr_col],
                df_clean[base_col]
            )
            df_clean = df_clean.drop(columns=[pfr_col])
        elif pfr_col in df_clean.columns:
            df_clean = df_clean.rename(columns={pfr_col: base_col})

    # ============================================================================
    # STEP 6: Consolidate receiving stats
    # ============================================================================

    receiving_stats = {
        'Rec': 'PFR_Rec',
        'Rec_Yds': 'PFR_Rec_Yds',
        'Rec_TD': 'PFR_Rec_TD'
    }

    for base_col, pfr_col in receiving_stats.items():
        if base_col in df_clean.columns and pfr_col in df_clean.columns:
            df_clean[base_col] = np.where(
                (df_clean[base_col].isna()) | (df_clean[base_col] == 0),
                df_clean[pfr_col],
                df_clean[base_col]
            )
            df_clean = df_clean.drop(columns=[pfr_col])
        elif pfr_col in df_clean.columns:
            df_clean = df_clean.rename(columns={pfr_col: base_col})

    # ============================================================================
    # STEP 7: Keep PFR_Age and PFR_Tgt (if they exist) - these are new columns
    # ============================================================================
    # These don't need consolidation - they're additional information

    # ============================================================================
    # STEP 8: Reorder columns for better readability
    # ============================================================================

    preferred_order = [
        'Year', 'Rank', 'Player', 'Team', 'Position', 'Votes',
        'PFR_Age', 'G', 'GS',
        # Passing
        'Pass_Cmp', 'Pass_Att', 'Pass_Yds', 'Pass_TD', 'Pass_Int', 'PFR_Rate',
        # Rushing
        'Rush_Att', 'Rush_Yds', 'Rush_TD',
        # Receiving
        'Rec', 'Rec_Yds', 'Rec_TD', 'PFR_Tgt'
    ]

    # Get columns that exist in preferred order
    final_columns = [c for c in preferred_order if c in df_clean.columns]

    # Add any remaining columns not in preferred order
    remaining_cols = [c for c in df_clean.columns if c not in final_columns]
    final_columns.extend(remaining_cols)

    df_clean = df_clean[final_columns]

    # ============================================================================
    # STEP 9: Convert all numeric columns to proper types
    # ============================================================================

    numeric_columns = [
        'Year', 'Rank', 'Votes', 'PFR_Age', 'G', 'GS',
        'Pass_Cmp', 'Pass_Att', 'Pass_Yds', 'Pass_TD', 'Pass_Int', 'PFR_Rate',
        'Rush_Att', 'Rush_Yds', 'Rush_TD',
        'Rec', 'Rec_Yds', 'Rec_TD', 'PFR_Tgt'
    ]

    for col in numeric_columns:
        if col in df_clean.columns:
            df_clean[col] = pd.to_numeric(df_clean[col], errors='coerce')

    # ============================================================================
    # STEP 10: Final cleanup
    # ============================================================================

    print(f"\nCleaned shape: {df_clean.shape}")
    print(f"Cleaned columns: {df_clean.columns.tolist()}")
    print(f"\nColumns removed: {set(df.columns) - set(df_clean.columns)}")
    print(f"\nSample of cleaned data:")
    print(df_clean.head())

    return df_clean


# ============================================================================
# USAGE
# ============================================================================

# Load your messy data
oroy_df_adv = pd.read_csv('oroy_df_adv.csv')

# Clean it
oroy_df_final = clean_oroy_advanced(oroy_df_adv)

# ============================================================================
# SAVE CLEANED DATA
# ============================================================================

# Save main cleaned file
oroy_df_final.to_csv('oroy_df_final_clean.csv', index=False)
print("\n✓ Saved cleaned data to: oroy_df_final_clean.csv")

# Save timestamped backup
from datetime import datetime
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
backup_filename = f'oroy_final_clean_{timestamp}.csv'
oroy_df_final.to_csv(backup_filename, index=False)
print(f"✓ Saved timestamped backup to: {backup_filename}")

# If running in Google Colab, download the files
try:
    from google.colab import files
    print("\n📥 Downloading files...")
    files.download('oroy_df_final_clean.csv')
    print("✓ Downloaded: oroy_df_final_clean.csv")
except:
    print("\n(Not in Colab environment - files saved locally)")

# ============================================================================
# DISPLAY SUMMARY
# ============================================================================

print("\n" + "="*60)
print("SUMMARY STATISTICS")
print("="*60)
print(f"Total players: {len(oroy_df_final)}")
print(f"Years covered: {oroy_df_final['Year'].min():.0f} - {oroy_df_final['Year'].max():.0f}")
print(f"Positions: {oroy_df_final['Position'].value_counts().to_dict()}")

# Show a few examples
print("\n" + "="*60)
print("SAMPLE CLEANED DATA (2024 Top 5)")
print("="*60)
print(oroy_df_final[oroy_df_final['Year'] == 2024].head()[
    ['Year', 'Rank', 'Player', 'Team', 'Position', 'Votes',
     'Pass_Yds', 'Rush_Yds', 'Rec_Yds']
])

print("\n" + "="*60)
print("="*60)

Starting advanced OROY data cleaning...
Original shape: (185, 33)
Original columns: ['Year', 'Rank', 'Player', 'Team', 'Position', 'Votes', 'G', 'GS', 'Pass_Cmp', 'Pass_Att', 'Pass_Yds', 'Pass_TD', 'Pass_Int', 'Rush_Att', 'Rush_Yds', 'Rush_TD', 'Rec', 'Rec_Yds', 'Rec_TD', 'PFR_Age', 'PFR_Tm', 'PFR_Pos', 'PFR_G', 'PFR_GS', 'PFR_Pass_Cmp', 'PFR_Pass_Int', 'PFR_Rate', 'PFR_Pos_rush_dup', 'PFR_Rush_Att', 'PFR_Rush_Yds', 'PFR_Rush_TD', 'PFR_Pos_rec_dup', 'PFR_Rec']

Position columns found: ['Position', 'PFR_Pos', 'PFR_Pos_rush_dup', 'PFR_Pos_rec_dup']

Cleaned shape: (185, 21)
Cleaned columns: ['Year', 'Rank', 'Player', 'Team', 'Position', 'Votes', 'PFR_Age', 'G', 'GS', 'Pass_Cmp', 'Pass_Att', 'Pass_Yds', 'Pass_TD', 'Pass_Int', 'PFR_Rate', 'Rush_Att', 'Rush_Yds', 'Rush_TD', 'Rec', 'Rec_Yds', 'Rec_TD']

Columns removed: {'PFR_Pos_rec_dup', 'PFR_Pass_Int', 'PFR_Pos', 'PFR_Rush_TD', 'PFR_Rec', 'PFR_Rush_Yds', 'PFR_Rush_Att', 'PFR_G', 'PFR_GS', 'PFR_Pos_rush_dup', 'PFR_Pass_Cmp', 'PFR_Tm'}

Sam

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

✓ Downloaded: oroy_df_final_clean.csv

SUMMARY STATISTICS
Total players: 185
Years covered: 1980 - 2024
Positions: {'RB': 55, 'WR': 42, 'QB': 32, 'rb': 9, 'TE': 7, 'FB': 7, 'RG': 6, 'LT': 5, 'RT': 4, 'K': 4, 'LG': 3, 'wr': 3, 'qb': 2, 'C': 2, 'OL': 1, 'T': 1, 'OT': 1, 'te': 1}

SAMPLE CLEANED DATA (2024 Top 5)
   Year  Rank          Player                   Team Position  Votes  \
0  2024     1  Jayden Daniels  Washington Commanders       QB  115.0   
1  2024     2    Brock Bowers      Las Vegas Raiders       TE   50.0   
2  2024     3          Bo Nix         Denver Broncos       QB   36.0   
3  2024     4    Brian Thomas   Jacksonville Jaguars       WR   19.0   
4  2024     5    Malik Nabers        New York Giants       WR   12.0   

   Pass_Yds  Rush_Yds  Rec_Yds  
0      3568     891.0        0  
1         0      13.0     1194  
2      3775     430.0        2  
3         0      48.0     1282  
4         0       2.0     1204  



In [1]:
# DEFENSIVE ROOKIE OF THE YEAR (DROY) SCRAPING AND CLEANING
# Complete code with all helper functions

import pandas as pd
import numpy as np
import time
from typing import Optional

# ============================================================================
# HELPER FUNCTIONS
# ============================================================================

def _clean_player_name(name: str) -> str:
    """Remove asterisks and plus signs from player names."""
    if pd.isna(name):
        return name
    name = str(name).strip()
    name = name.replace('*', '').replace('+', '')
    return name.strip()

def _pick_main_table(tables: list) -> Optional[pd.DataFrame]:
    """Pick the largest table from a list of tables."""
    if not tables:
        return None
    largest = max(tables, key=lambda t: t.shape[0] * t.shape[1])
    return largest

def _flatten_cols(df: pd.DataFrame) -> pd.DataFrame:
    """Flatten multi-level columns to single level."""
    if isinstance(df.columns, pd.MultiIndex):
        df.columns = [' '.join(str(c).strip() for c in col if str(c) != 'nan').strip()
                      for col in df.columns]
    return df

def _prefer_totals_rows(df: pd.DataFrame) -> pd.DataFrame:
    """
    For players with multiple rows (multiple teams), prefer the row with Tm='TOT' if available.
    Otherwise, keep the first row.
    """
    if 'Tm' not in df.columns or 'Player' not in df.columns:
        return df

    df = df.copy()

    # Group by Player and Year
    if 'Year' in df.columns:
        groups = df.groupby(['Player', 'Year'])
    else:
        groups = df.groupby('Player')

    result_rows = []
    for name, group in groups:
        if len(group) == 1:
            result_rows.append(group.iloc[0])
        else:
            # Check for TOT row
            tot_rows = group[group['Tm'] == 'TOT']
            if not tot_rows.empty:
                result_rows.append(tot_rows.iloc[0])
            else:
                result_rows.append(group.iloc[0])

    return pd.DataFrame(result_rows).reset_index(drop=True)

# ============================================================================
# STEP 1: SCRAPE DROY DATA
# ============================================================================

def scrape_droy_data(start_year=2024, end_year=1980):
    """Scrape DROY voting data from Pro Football Reference."""
    print("="*60)
    print("SCRAPING DROY DATA")
    print("="*60)

    year = start_year
    droy_dfs = []

    while year >= end_year:
        url = f'https://www.pro-football-reference.com/awards/awards_{year}.htm'
        try:
            tables = pd.read_html(url)
            droy_table = tables[4]  # DROY is typically table index 4

            if isinstance(droy_table.columns, pd.MultiIndex):
                droy_table.columns = [' '.join(str(col).strip() for col in multi_col if str(col) != 'nan')
                                       for multi_col in droy_table.columns]

            droy_table['Year'] = year
            droy_dfs.append(droy_table)
            print(f"Scraped DROY {year} - Shape: {droy_table.shape}")

        except Exception as e:
            print(f"Failed to scrape DROY {year}: {e}")

        time.sleep(3)
        year -= 1

    droy_df = pd.concat(droy_dfs, ignore_index=True)
    print(f"\nDROY Total rows: {len(droy_df)}")
    print(f"Years covered: {droy_df['Year'].min()} - {droy_df['Year'].max()}")

    return droy_df

# ============================================================================
# STEP 2: CLEAN DROY DATA
# ============================================================================

def clean_droy_data(droy_df):
    """Clean the scraped DROY DataFrame."""
    print("\nStarting DROY data cleaning...")
    print(f"Original shape: {droy_df.shape}")

    droy_df_clean = droy_df.copy()

    # Drop unnecessary columns
    droy_df_clean = droy_df_clean.drop(columns=["Unnamed: 0"], errors="ignore")

    # Rename columns
    column_mapping = {
        "Unnamed: 0_level_0 Rk": "Rank",
        "Unnamed: 1_level_0 Pos": "Position",
        "Unnamed: 2_level_0 Player": "Player",
        "Unnamed: 3_level_0 Tm": "Team",
        "Unnamed: 4_level_0 Vote Pts": "VotePoints_1",
        "Unnamed: 4_level_0 Votes": "VotePoints_2",
        "Unnamed: 5_level_0 1st Place": "FirstPlaceVotes",
        "Unnamed: 6_level_0 Share": "VoteShare_1",
        "Unnamed: 5_level_0 Share": "VoteShare_2",
        "Games G": "G",
        "Games GS": "GS",
        # Defensive stats
        "Defense Int": "Def_Int",
        "Defense Sk": "Def_Sk",
        "Defense Comb": "Def_Comb",
        "Defense Solo": "Def_Solo",
        "Defense Ast": "Def_Ast",
        "Defense TFL": "Def_TFL",
        "Defense QBHits": "Def_QBHits",
        "Defense FR": "Def_FR",
        "Defense FF": "Def_FF",
        "Defense TD": "Def_TD",
        "Defense PD": "Def_PD",
    }

    droy_df_clean = droy_df_clean.rename(columns=column_mapping)

    # Consolidate voting columns
    vote_points_cols = [c for c in ["VotePoints_1", "VotePoints_2"] if c in droy_df_clean.columns]
    if vote_points_cols:
        droy_df_clean["VotePoints"] = pd.to_numeric(droy_df_clean[vote_points_cols[0]], errors="coerce")
        for col in vote_points_cols[1:]:
            droy_df_clean["VotePoints"] = droy_df_clean["VotePoints"].fillna(
                pd.to_numeric(droy_df_clean[col], errors="coerce")
            )
        droy_df_clean = droy_df_clean.drop(columns=vote_points_cols)

    # Convert FirstPlaceVotes to numeric
    if "FirstPlaceVotes" in droy_df_clean.columns:
        droy_df_clean["FirstPlaceVotes"] = pd.to_numeric(droy_df_clean["FirstPlaceVotes"], errors="coerce")

    # Create unified Votes column
    if "VotePoints" in droy_df_clean.columns and "Year" in droy_df_clean.columns:
        votes_2022_plus = droy_df_clean["VotePoints"] / 4.2
        votes_2022_plus = np.where(pd.isna(votes_2022_plus), np.nan, np.round(votes_2022_plus))
        votes_2022_plus = np.where(
            (votes_2022_plus == 0) & (droy_df_clean["VotePoints"] > 0),
            1,
            votes_2022_plus
        )

        droy_df_clean["Votes"] = np.where(
            droy_df_clean["Year"] >= 2022,
            votes_2022_plus,
            droy_df_clean["VotePoints"]
        )

    # Convert stat columns to numeric
    numeric_columns = [
        "Rank", "VotePoints", "FirstPlaceVotes", "Votes",
        "G", "GS", "Year",
        "Def_Int", "Def_Sk", "Def_Comb", "Def_Solo", "Def_Ast",
        "Def_TFL", "Def_QBHits", "Def_FR", "Def_FF", "Def_TD", "Def_PD"
    ]

    for col in numeric_columns:
        if col in droy_df_clean.columns:
            droy_df_clean[col] = pd.to_numeric(droy_df_clean[col], errors="coerce")

    # Clean text columns
    for col in ["Position", "Player", "Team"]:
        if col in droy_df_clean.columns:
            droy_df_clean[col] = droy_df_clean[col].astype(str).str.strip()

    # Drop unwanted columns
    cols_to_drop = ["VoteShare_1", "VoteShare_2", "VotePoints", "FirstPlaceVotes"]
    droy_df_clean = droy_df_clean.drop(
        columns=[c for c in cols_to_drop if c in droy_df_clean.columns],
        errors="ignore"
    )

    # Reorder columns
    preferred_order = [
        "Year", "Rank", "Player", "Team", "Position", "Votes",
        "G", "GS",
        "Def_Comb", "Def_Solo", "Def_Ast", "Def_TFL", "Def_Sk",
        "Def_QBHits", "Def_Int", "Def_PD", "Def_FR", "Def_FF", "Def_TD"
    ]

    final_columns = [c for c in preferred_order if c in droy_df_clean.columns]
    final_columns.extend([c for c in droy_df_clean.columns if c not in final_columns])
    droy_df_clean = droy_df_clean[final_columns]

    print(f"Cleaned shape: {droy_df_clean.shape}")
    print(f"Columns: {droy_df_clean.columns.tolist()}")

    return droy_df_clean

# ============================================================================
# STEP 3: FETCH PFR DEFENSIVE STATS
# ============================================================================

def fetch_defense(year: int, max_retries: int = 3) -> Optional[pd.DataFrame]:
    """Fetch defensive stats from Pro Football Reference."""
    url = f'https://www.pro-football-reference.com/years/{year}/defense.htm'

    for attempt in range(max_retries):
        try:
            tables = pd.read_html(url)
            t = _pick_main_table(tables)
            if t is None:
                return None

            t = _flatten_cols(t)

            column_map = {}
            for col in t.columns:
                col_str = str(col)
                if 'Player' in col_str:
                    column_map[col] = 'Player'
                elif col_str in ['Team', 'Tm']:
                    column_map[col] = 'Tm'
                elif col_str == 'Age':
                    column_map[col] = 'Age'
                elif 'Pos' in col_str:
                    column_map[col] = 'Pos'
                elif col_str == 'G':
                    column_map[col] = 'G'
                elif col_str == 'GS':
                    column_map[col] = 'GS'
                elif 'Int' in col_str and 'Defense' in col_str:
                    column_map[col] = 'Def_Int'
                elif 'Sk' in col_str and 'Defense' in col_str:
                    column_map[col] = 'Def_Sk'
                elif 'Comb' in col_str or ('Tackles Comb' in col_str):
                    column_map[col] = 'Def_Comb'
                elif 'Solo' in col_str or ('Tackles Solo' in col_str):
                    column_map[col] = 'Def_Solo'
                elif 'Ast' in col_str or ('Tackles Ast' in col_str):
                    column_map[col] = 'Def_Ast'
                elif 'TFL' in col_str:
                    column_map[col] = 'Def_TFL'
                elif 'QBHits' in col_str or 'QB Hits' in col_str:
                    column_map[col] = 'Def_QBHits'
                elif 'FR' in col_str and 'Fumbles' in col_str:
                    column_map[col] = 'Def_FR'
                elif 'FF' in col_str and 'Fumbles' in col_str:
                    column_map[col] = 'Def_FF'
                elif 'TD' in col_str and 'Defense' in col_str:
                    column_map[col] = 'Def_TD'
                elif 'PD' in col_str or 'Pass Def' in col_str:
                    column_map[col] = 'Def_PD'

            if 'Player' not in column_map.values():
                return None

            cols_to_keep = list(column_map.keys())
            t = t[cols_to_keep].copy()
            t = t.rename(columns=column_map)

            if t.columns.duplicated().any():
                t = t.loc[:, ~t.columns.duplicated()]

            t['Year'] = year
            t['Player'] = t['Player'].apply(_clean_player_name)
            t = _prefer_totals_rows(t)

            rename_map = {col: f'PFR_{col}' for col in t.columns if col not in ['Player', 'Year']}
            t = t.rename(columns=rename_map)

            return t

        except Exception as e:
            if attempt < max_retries - 1:
                time.sleep(2)
            else:
                return None

# ============================================================================
# STEP 4: MERGE DROY WITH PFR STATS
# ============================================================================

def merge_droy_with_pfr(droy_df_clean: pd.DataFrame, sleep_time: float = 3.0) -> pd.DataFrame:
    """Merge DROY data with PFR defensive stats."""

    if 'Player' not in droy_df_clean.columns or 'Year' not in droy_df_clean.columns:
        raise ValueError("droy_df_clean must contain 'Player' and 'Year' columns")

    droy_df_aug = droy_df_clean.copy()
    droy_df_aug['Player_clean'] = droy_df_aug['Player'].apply(_clean_player_name)

    years = sorted(droy_df_aug['Year'].dropna().astype(int).unique().tolist(), reverse=True)
    print(f"\nProcessing {len(years)} years: {min(years)} - {max(years)}")

    def_frames = []

    for i, year in enumerate(years):
        print(f"Year {year} ({i+1}/{len(years)})...", end=' ')

        def_data = fetch_defense(year)
        if def_data is not None:
            def_data['Player_clean'] = def_data['Player']
            def_frames.append(def_data)
            print("✓ Defense")
        else:
            print("✗ Failed")

        time.sleep(sleep_time)

    # Combine all data
    def_all = pd.concat(def_frames, ignore_index=True) if def_frames else pd.DataFrame()

    print(f"\nCombined defensive data: {def_all.shape}")

    # Merge with DROY data
    merged = droy_df_aug.copy()

    if not def_all.empty:
        merged = merged.merge(
            def_all.drop(columns=['Player'], errors='ignore'),
            on=['Year', 'Player_clean'],
            how='left',
            suffixes=('', '_dup')
        )

    merged = merged.drop(columns=['Player_clean'])

    # Convert numeric columns
    for col in merged.columns:
        if 'PFR_' in col:
            merged[col] = pd.to_numeric(merged[col], errors='ignore')

    print(f"\nFinal merged shape: {merged.shape}")
    return merged

# ============================================================================
# CLEAN ADVANCED DROY DATA (remove duplicates)
# ============================================================================

def clean_droy_advanced(df):
    """
    Clean the DROY advanced dataset by removing duplicate columns
    and consolidating PFR stats with voting stats.
    """
    print("\nStarting advanced DROY data cleaning...")
    print(f"Original shape: {df.shape}")

    df_clean = df.copy()

    # Consolidate Position columns
    pos_cols = [c for c in df_clean.columns if 'Pos' in c]
    if 'Position' in df_clean.columns:
        df_clean['Position'] = df_clean['Position'].fillna(df_clean.get('PFR_Pos', ''))
    elif 'PFR_Pos' in df_clean.columns:
        df_clean['Position'] = df_clean['PFR_Pos']

    df_clean = df_clean.drop(columns=[c for c in pos_cols if c != 'Position' and c in df_clean.columns])

    # Consolidate team columns
    if 'Team' in df_clean.columns and 'PFR_Tm' in df_clean.columns:
        df_clean['Team'] = df_clean['Team'].fillna(df_clean['PFR_Tm'])
        df_clean = df_clean.drop(columns=['PFR_Tm'])
    elif 'PFR_Tm' in df_clean.columns:
        df_clean = df_clean.rename(columns={'PFR_Tm': 'Team'})

    # Consolidate games columns
    if 'G' in df_clean.columns and 'PFR_G' in df_clean.columns:
        df_clean['G'] = df_clean['G'].fillna(df_clean['PFR_G'])
        df_clean = df_clean.drop(columns=['PFR_G'])
    elif 'PFR_G' in df_clean.columns:
        df_clean = df_clean.rename(columns={'PFR_G': 'G'})

    if 'GS' in df_clean.columns and 'PFR_GS' in df_clean.columns:
        df_clean['GS'] = df_clean['GS'].fillna(df_clean['PFR_GS'])
        df_clean = df_clean.drop(columns=['PFR_GS'])
    elif 'PFR_GS' in df_clean.columns:
        df_clean = df_clean.rename(columns={'PFR_GS': 'GS'})

    # Consolidate defensive stats
    def_stats = {
        'Def_Int': 'PFR_Def_Int',
        'Def_Sk': 'PFR_Def_Sk',
        'Def_Comb': 'PFR_Def_Comb',
        'Def_Solo': 'PFR_Def_Solo',
        'Def_Ast': 'PFR_Def_Ast',
        'Def_TFL': 'PFR_Def_TFL',
        'Def_QBHits': 'PFR_Def_QBHits',
        'Def_FR': 'PFR_Def_FR',
        'Def_FF': 'PFR_Def_FF',
        'Def_TD': 'PFR_Def_TD',
        'Def_PD': 'PFR_Def_PD'
    }

    for base_col, pfr_col in def_stats.items():
        if base_col in df_clean.columns and pfr_col in df_clean.columns:
            df_clean[base_col] = np.where(
                (df_clean[base_col].isna()) | (df_clean[base_col] == 0),
                df_clean[pfr_col],
                df_clean[base_col]
            )
            df_clean = df_clean.drop(columns=[pfr_col])
        elif pfr_col in df_clean.columns:
            df_clean = df_clean.rename(columns={pfr_col: base_col})

    # Reorder columns
    preferred_order = [
        'Year', 'Rank', 'Player', 'Team', 'Position', 'Votes',
        'PFR_Age', 'G', 'GS',
        'Def_Comb', 'Def_Solo', 'Def_Ast', 'Def_TFL', 'Def_Sk',
        'Def_QBHits', 'Def_Int', 'Def_PD', 'Def_FR', 'Def_FF', 'Def_TD'
    ]

    final_columns = [c for c in preferred_order if c in df_clean.columns]
    remaining_cols = [c for c in df_clean.columns if c not in final_columns]
    final_columns.extend(remaining_cols)

    df_clean = df_clean[final_columns]

    # Convert numeric columns
    numeric_columns = [
        'Year', 'Rank', 'Votes', 'PFR_Age', 'G', 'GS',
        'Def_Comb', 'Def_Solo', 'Def_Ast', 'Def_TFL', 'Def_Sk',
        'Def_QBHits', 'Def_Int', 'Def_PD', 'Def_FR', 'Def_FF', 'Def_TD'
    ]

    for col in numeric_columns:
        if col in df_clean.columns:
            df_clean[col] = pd.to_numeric(df_clean[col], errors='coerce')

    print(f"Cleaned shape: {df_clean.shape}")
    print(f"Cleaned columns: {df_clean.columns.tolist()}")

    return df_clean

# ============================================================================
# MAIN EXECUTION
# ============================================================================

if __name__ == "__main__":
    # Step 1: Scrape
    droy_df = scrape_droy_data(start_year=2024, end_year=1980)
    droy_df.to_csv('droy_df.csv', index=False)

    # Step 2: Clean
    droy_df_clean = clean_droy_data(droy_df)
    droy_df_clean.to_csv('droy_df_clean.csv', index=False)

    # Step 3: Merge with PFR stats
    droy_df_adv = merge_droy_with_pfr(droy_df_clean, sleep_time=3)
    droy_df_adv.to_csv('droy_df_adv.csv', index=False)

    # Step 4: Clean advanced data (remove duplicates)
    droy_df_final = clean_droy_advanced(droy_df_adv)

    # ============================================================================
    # SAVE CLEANED DATA
    # ============================================================================

    # Save main cleaned file
    droy_df_final.to_csv('droy_df_final_clean.csv', index=False)
    print("\n✓ Saved cleaned data to: droy_df_final_clean.csv")

    # Save timestamped backup
    from datetime import datetime
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    backup_filename = f'droy_final_clean_{timestamp}.csv'
    droy_df_final.to_csv(backup_filename, index=False)
    print(f"✓ Saved timestamped backup to: {backup_filename}")

    # If running in Google Colab, download the files
    try:
        from google.colab import files
        print("\n📥 Downloading files...")
        files.download('droy_df_final_clean.csv')
        print("✓ Downloaded: droy_df_final_clean.csv")
    except:
        print("\n(Not in Colab environment - files saved locally)")

    # ============================================================================
    # DISPLAY SUMMARY
    # ============================================================================

    print("\n" + "="*60)
    print("SUMMARY STATISTICS")
    print("="*60)
    print(f"Total players: {len(droy_df_final)}")
    print(f"Years covered: {droy_df_final['Year'].min():.0f} - {droy_df_final['Year'].max():.0f}")
    print(f"Positions: {droy_df_final['Position'].value_counts().to_dict()}")

    # Show a few examples
    print("\n" + "="*60)
    print("SAMPLE CLEANED DATA (2024 Top 5)")
    print("="*60)
    print(droy_df_final[droy_df_final['Year'] == 2024].head()[
        ['Year', 'Rank', 'Player', 'Team', 'Position', 'Votes',
         'Def_Comb', 'Def_Sk', 'Def_Int']
    ])

    print("\n" + "="*60)
    print("")
    print("="*60)

SCRAPING DROY DATA
Scraped DROY 2024 - Shape: (16, 16)
Scraped DROY 2023 - Shape: (10, 16)
Scraped DROY 2022 - Shape: (6, 16)
Scraped DROY 2021 - Shape: (1, 15)
Scraped DROY 2020 - Shape: (4, 15)
Scraped DROY 2019 - Shape: (4, 15)
Scraped DROY 2018 - Shape: (3, 15)
Scraped DROY 2017 - Shape: (3, 15)
Scraped DROY 2016 - Shape: (3, 15)
Scraped DROY 2015 - Shape: (3, 15)
Scraped DROY 2014 - Shape: (4, 15)
Scraped DROY 2013 - Shape: (5, 15)
Scraped DROY 2012 - Shape: (5, 15)
Scraped DROY 2011 - Shape: (2, 15)
Scraped DROY 2010 - Shape: (2, 15)
Scraped DROY 2009 - Shape: (4, 15)
Scraped DROY 2008 - Shape: (2, 15)
Scraped DROY 2007 - Shape: (2, 15)
Scraped DROY 2006 - Shape: (7, 15)
Scraped DROY 2005 - Shape: (4, 15)
Scraped DROY 2004 - Shape: (8, 15)
Scraped DROY 2003 - Shape: (6, 15)
Scraped DROY 2002 - Shape: (5, 15)
Scraped DROY 2001 - Shape: (5, 15)
Scraped DROY 2000 - Shape: (4, 15)
Scraped DROY 1999 - Shape: (2, 15)
Scraped DROY 1998 - Shape: (3, 15)
Scraped DROY 1997 - Shape: (6, 15)

/tmp/ipython-input-753113697.py:350: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  merged[col] = pd.to_numeric(merged[col], errors='ignore')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

✓ Downloaded: droy_df_final_clean.csv

SUMMARY STATISTICS
Total players: 229
Years covered: 1980 - 2024
Positions: {'LCB': 20, 'FS': 17, 'LDE': 16, 'LILB': 15, 'RCB': 14, 'MLB': 13, 'RDE': 12, 'CB': 10, 'ROLB': 10, 'RLB': 10, 'DE': 10, 'LLB': 9, 'LB': 9, 'RILB': 8, 'LDT': 8, 'SS': 7, 'LOLB': 6, 'DB': 5, 'NT': 5, 'RDT': 4, 'S': 3, 'DT-DE': 2, 'OLB': 1, 'lilb': 1, 'DL': 1, 'DT': 1, 'EDGE': 1, 'rcb': 1, 'DB-LB': 1, 'rlb': 1, 'LB-DE': 1, 'lolb': 1, 'rolb': 1, 'ss': 1, 'mlb': 1, 'DE-LB-DT-NT': 1, 'lcb': 1, 'NT-DT-DE': 1}

SAMPLE CLEANED DATA (2024 Top 5)


KeyError: "['Def_Sk', 'Def_Int'] not in index"

In [4]:
"""
DROY DATA CLEANING SCRIPT (STANDALONE)
Use this to clean an existing DROY CSV file without needing to rescrape.
"""

import pandas as pd
import numpy as np

def clean_droy_file(input_file='droy_df_final_clean.csv', output_file='droy_cleaned.csv'):
    """
    Clean a DROY CSV file by removing duplicate columns and standardizing the data.

    Parameters:
    -----------
    input_file : str
        Path to the input CSV file
    output_file : str
        Path to save the cleaned CSV file
    """

    print("="*60)
    print("DROY DATA CLEANING")
    print("="*60)

    # Load the data
    print(f"\nLoading data from: {input_file}")
    df = pd.read_csv(input_file)
    print(f"Original shape: {df.shape}")
    print(f"Original columns: {df.columns.tolist()}")

    df_clean = df.copy()

    # ========================================================================
    # STEP 1: Remove obvious duplicate columns
    # ========================================================================
    print("\n--- Step 1: Removing duplicate columns ---")

    # Identify columns to drop (duplicates with different names)
    columns_to_drop = []

    # Drop "Tackles Solo" and "Tackles Ast" if we have Def_Solo and Def_Ast
    if 'Tackles Solo' in df_clean.columns and 'Def_Solo' in df_clean.columns:
        columns_to_drop.append('Tackles Solo')
    if 'Tackles Ast' in df_clean.columns and 'Def_Ast' in df_clean.columns:
        columns_to_drop.append('Tackles Ast')

    # Drop unnamed columns
    unnamed_cols = [c for c in df_clean.columns if 'Unnamed' in str(c)]
    columns_to_drop.extend(unnamed_cols)

    # Drop "Def Interceptions" variations if we have Def_Int
    interception_cols = [c for c in df_clean.columns
                        if 'Def Interceptions' in str(c) and c != 'Def_Int']
    columns_to_drop.extend(interception_cols)

    # Remove duplicates from the list
    columns_to_drop = list(set(columns_to_drop))

    print(f"Dropping {len(columns_to_drop)} duplicate columns: {columns_to_drop}")
    df_clean = df_clean.drop(columns=columns_to_drop, errors='ignore')

    # ========================================================================
    # STEP 2: Consolidate defensive statistics
    # ========================================================================
    print("\n--- Step 2: Consolidating defensive statistics ---")

    # Map of final column names to possible source columns (in priority order)
    stat_consolidation = {
        'Def_Sk': ['Def_Sk', 'Sk', 'Unnamed: 11_level_0 Sk', 'Unnamed: 10_level_0 Sk', 'Unnamed: 8_level_0 Sk'],
        'Def_Int': ['Def_Int', 'Int'],
        'Def_Comb': ['Def_Comb', 'Comb'],
        'Def_Solo': ['Def_Solo', 'Solo'],
        'Def_Ast': ['Def_Ast', 'Ast'],
        'Def_TFL': ['Def_TFL', 'TFL'],
        'Def_QBHits': ['Def_QBHits', 'QBHits', 'QB Hits'],
        'Def_FR': ['Def_FR', 'FR'],
        'Def_FF': ['Def_FF', 'FF'],
        'Def_TD': ['Def_TD', 'TD'],
        'Def_PD': ['Def_PD', 'PD']
    }

    for final_col, source_cols in stat_consolidation.items():
        # Find which source columns exist
        existing_sources = [c for c in source_cols if c in df_clean.columns]

        if not existing_sources:
            continue

        # If the final column doesn't exist, use the first available source
        if final_col not in df_clean.columns:
            df_clean[final_col] = df_clean[existing_sources[0]]
            print(f"Created {final_col} from {existing_sources[0]}")
        else:
            # Fill NaN values in the final column with values from other sources
            for source in existing_sources[1:]:
                mask = df_clean[final_col].isna() | (df_clean[final_col] == 0)
                df_clean.loc[mask, final_col] = df_clean.loc[mask, source]

        # Drop the source columns (except the final one)
        cols_to_drop = [c for c in existing_sources if c != final_col]
        df_clean = df_clean.drop(columns=cols_to_drop, errors='ignore')

    # ========================================================================
    # STEP 3: Convert all numeric columns to proper types
    # ========================================================================
    print("\n--- Step 3: Converting numeric columns ---")

    numeric_columns = [
        'Year', 'Rank', 'Votes', 'PFR_Age', 'G', 'GS',
        'Def_Comb', 'Def_Solo', 'Def_Ast', 'Def_TFL', 'Def_Sk',
        'Def_QBHits', 'Def_Int', 'Def_PD', 'Def_FR', 'Def_FF', 'Def_TD'
    ]

    for col in numeric_columns:
        if col in df_clean.columns:
            df_clean[col] = pd.to_numeric(df_clean[col], errors='coerce')

    # ========================================================================
    # STEP 4: Clean text columns
    # ========================================================================
    print("\n--- Step 4: Cleaning text columns ---")

    text_columns = ['Player', 'Team', 'Position']
    for col in text_columns:
        if col in df_clean.columns:
            df_clean[col] = df_clean[col].astype(str).str.strip()
            df_clean[col] = df_clean[col].replace('nan', '')

    # ========================================================================
    # STEP 5: Reorder columns for clarity
    # ========================================================================
    print("\n--- Step 5: Reordering columns ---")

    preferred_order = [
        'Year', 'Rank', 'Player', 'Team', 'Position', 'Votes',
        'PFR_Age', 'G', 'GS',
        'Def_Comb', 'Def_Solo', 'Def_Ast', 'Def_TFL', 'Def_Sk',
        'Def_QBHits', 'Def_Int', 'Def_PD', 'Def_FR', 'Def_FF', 'Def_TD'
    ]

    # Build final column list
    final_columns = [c for c in preferred_order if c in df_clean.columns]
    remaining_cols = [c for c in df_clean.columns if c not in final_columns]
    final_columns.extend(remaining_cols)

    df_clean = df_clean[final_columns]

    # ========================================================================
    # STEP 6: Save cleaned data
    # ========================================================================
    print(f"\n--- Step 6: Saving cleaned data ---")
    print(f"Cleaned shape: {df_clean.shape}")
    print(f"Cleaned columns: {df_clean.columns.tolist()}")

    df_clean.to_csv(output_file, index=False)
    print(f"\n✓ Saved cleaned data to: {output_file}")

    # ========================================================================
    # DISPLAY SUMMARY
    # ========================================================================
    print("\n" + "="*60)
    print("SUMMARY STATISTICS")
    print("="*60)
    print(f"Total players: {len(df_clean)}")
    print(f"Years covered: {df_clean['Year'].min():.0f} - {df_clean['Year'].max():.0f}")
    print(f"\nColumns in cleaned file: {len(df_clean.columns)}")
    print(f"Rows in cleaned file: {len(df_clean)}")

    # Check for positions
    if 'Position' in df_clean.columns:
        print(f"\nTop positions:")
        print(df_clean['Position'].value_counts().head(10))

    # Show sample of recent data
    print("\n" + "="*60)
    print("SAMPLE CLEANED DATA (Most Recent Year, Top 5)")
    print("="*60)
    recent_year = df_clean['Year'].max()
    sample_cols = ['Year', 'Rank', 'Player', 'Team', 'Position', 'Votes',
                   'Def_Comb', 'Def_Sk', 'Def_Int']
    available_cols = [c for c in sample_cols if c in df_clean.columns]
    print(df_clean[df_clean['Year'] == recent_year].head()[available_cols])

    print("\n" + "="*60)
    print("CLEANING COMPLETE!")
    print("="*60)

    return df_clean


# ============================================================================
# MAIN EXECUTION
# ============================================================================

if __name__ == "__main__":
    # Clean the file
    df_cleaned = clean_droy_file(
        input_file='droy_df_final_clean.csv',  # Change this to your input file name
        output_file='droy_cleaned.csv'          # Change this to your desired output file name
    )

    # Optional: Try to download in Colab
    try:
        from google.colab import files
        print("\n📥 Downloading cleaned file...")
        files.download('droy_cleaned.csv')
        print("✓ Downloaded: droy_cleaned.csv")
    except:
        print("\n(Not in Colab environment - file saved locally)")

DROY DATA CLEANING

Loading data from: droy_df_final_clean.csv
Original shape: (229, 24)
Original columns: ['Year', 'Rank', 'Player', 'Team', 'Position', 'Votes', 'G', 'GS', 'Def_Comb', 'Def_Solo', 'Def_Ast', 'Def_TFL', 'Def_QBHits', 'Def_PD', 'Def_FR', 'Def_FF', 'Tackles Solo', 'Tackles Ast', 'Unnamed: 11_level_0 Sk', 'Def Interceptions Int', 'Def Interceptions Yds', 'Def Interceptions TD', 'Unnamed: 10_level_0 Sk', 'Unnamed: 8_level_0 Sk']

--- Step 1: Removing duplicate columns ---
Dropping 8 duplicate columns: ['Def Interceptions Int', 'Def Interceptions Yds', 'Def Interceptions TD', 'Tackles Solo', 'Tackles Ast', 'Unnamed: 8_level_0 Sk', 'Unnamed: 11_level_0 Sk', 'Unnamed: 10_level_0 Sk']

--- Step 2: Consolidating defensive statistics ---

--- Step 3: Converting numeric columns ---

--- Step 4: Cleaning text columns ---

--- Step 5: Reordering columns ---

--- Step 6: Saving cleaned data ---
Cleaned shape: (229, 16)
Cleaned columns: ['Year', 'Rank', 'Player', 'Team', 'Position', 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

✓ Downloaded: droy_cleaned.csv
